In [5]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [4]:
import pandas as pd
import numpy as np
import os
import csv
import re
import sys
from typing import Optional

# ====================================================================
# 0. CONFIGURATION (THE ONLY LINE YOU MUST VERIFY)
# ====================================================================
# This MUST be the absolute path to the FOLDER containing your AACT .txt files.
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
# --------------------------------------------------------------------

print(f"DEBUG: Data Path is set to: {DATA_PATH}")
print("--- STARTING PREDICTION COHORT GENERATION ---")

# ====================================================================
# 1. CLINICAL TRIAL LOADER CLASS (Full Definition)
# ====================================================================

class ClinicalTrialLoader:
    """
    A specialized loader for generating the PROSPECTIVE prediction dataset (2024-2025).
    It ensures feature consistency and excludes all post-hoc/leakage features.
    """
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.df_drugs = pd.DataFrame()

        # --- STRATEGY A: PERFECT ---
        self.params_perfect = {
            "sep": "|", "dtype": str, "header": 0, "quotechar": '"',
            "quoting": csv.QUOTE_MINIMAL, "low_memory": False, "on_bad_lines": "warn"
        }

        # --- STRATEGY B: ROBUST ---
        self.params_robust = {
            "sep": "|", "dtype": str, "header": 0, "quotechar": '"',
            "quoting": 3, "low_memory": False, "on_bad_lines": "warn"
        }

    def _safe_load(self, filename: str, cols: Optional[list] = None) -> pd.DataFrame:
        full_path = os.path.join(self.data_path, filename)
        if not os.path.exists(full_path):
            print(f"   [!] Warning: File not found {filename}. Features will be empty.")
            return pd.DataFrame()

        try:
            return pd.read_csv(full_path, usecols=cols, **self.params_perfect)
        except Exception as e:
            print(f"   [!] Formatting error in {filename}. Switching to Robust Mode...")
            try:
                return pd.read_csv(full_path, usecols=cols, **self.params_robust)
            except Exception as e2:
                print(f"   [x] CRITICAL: Could not load {filename}. Error: {e2}")
                return pd.DataFrame()

    # --- STEP 1: LOAD FULL COHORT FOR TIME-SENSITIVE FEATURES ---

    def load_full_cohort_for_competition(self, min_year: int = 2005, max_year: int = 2025) -> pd.DataFrame:
        """
        Loads the entire relevant dataset (2005-2025) to calculate time-sensitive
        features (Competition) on the complete historical context before splitting.
        """
        print(f">>> 1. Loading Full Cohort ({min_year}-{max_year}) for Competition Calculation...")

        # 1. Load Core Columns
        cols_studies = ['nct_id', 'overall_status', 'study_type', 'phase',
                        'start_date', 'number_of_arms',
                        'official_title', 'why_stopped',
                        'has_dmc', 'is_fda_regulated_drug', 'brief_title']

        df = self._safe_load('studies.txt', cols=cols_studies)
        if df.empty:
            raise ValueError("Critical Error: 'studies.txt' failed to load.")

        # 2. Apply Core Filters (Interventional, Drug, Phase 2/3)
        df = df[df['study_type'].str.upper() == 'INTERVENTIONAL'].copy()

        cols_int = ['nct_id', 'intervention_type', 'name']
        df_int = self._safe_load('interventions.txt', cols=cols_int)
        if not df_int.empty:
            target_types = ['DRUG', 'BIOLOGICAL', 'GENETIC']
            drug_ids = df_int[df_int['intervention_type'].str.upper().isin(target_types)]['nct_id'].unique()
            df = df[df['nct_id'].isin(drug_ids)]
            self.df_drugs = df_int[df_int['intervention_type'].str.upper().isin(target_types + ['DIETARY SUPPLEMENT', 'OTHER'])].copy()

        excluded_phases = ['EARLY_PHASE1', 'PHASE4', 'NA'] #'PHASE1'
        df = df[~df['phase'].astype(str).str.upper().isin(excluded_phases)]
        df = df.dropna(subset=['phase'])

        # 3. Apply Full Date Range Filter (2005-2025)
        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        df['start_year'] = df['start_date'].dt.year
        df = df[df['start_year'].between(min_year, max_year)].copy()

        # 4. Calculate Time-Sensitive Features (Hierarchy and Competition)
        df = self._attach_medical_hierarchy(df)
        df = self._calculate_competition(df)

        print(f"    Full Cohort Size: {len(df)} trials.")
        return df.copy()


    # --- STEP 2: FILTER AND FINALIZE PREDICTION SET ---

    def load_and_clean_prediction_set(self, df_full_cohort: pd.DataFrame, start_year_predict: int = 2024) -> pd.DataFrame:
        """
        Filters the full cohort for the Prediction Set: Active/Pending Statuses, Year >= 2024.
        """
        df = df_full_cohort.copy()
        print(f">>> 2. Filtering for Prediction Set ({start_year_predict}-2025, Active/Pending Statuses)...")

        # 1. Filter: Date Range (2024-2025)
        df = df[df['start_year'] >= start_year_predict].copy()

        # 2. Filter: Status (Active/Pending Only)
        ACTIVE_STATUSES = [
            'RECRUITING', 'NOT_YET_RECRUITING', 'ACTIVE_NOT_RECRUITING',
            'ENROLLING_BY_INVITATION', 'WITHHELD', 'UNKNOWN'
        ]

        df_upper = df['overall_status'].astype(str).str.upper()
        df = df[df_upper.isin(ACTIVE_STATUSES)].copy()

        # 3. Filter: COVID Sanitizer (Kept for consistency)
        if 'why_stopped' in df.columns:
            covid_keywords = ['covid', 'pandemic', 'coronavirus', 'sars-cov-2', 'logistical reasons']
            mask_covid = df['why_stopped'].fillna('').astype(str).str.lower().apply(
                lambda x: any(k in x for k in covid_keywords)
            )
            if mask_covid.sum() > 0:
                print(f"    [Sanitizer] Dropping {mask_covid.sum()} trials terminated due to COVID/Logistics.")
                df = df[~mask_covid]

        # 4. Target: The prediction set has NO target.
        if 'target' in df.columns:
             df.drop(columns=['target'], inplace=True)

        print(f"    Prediction Cohort: {len(df)} trials (Active/Pending Phase 2/3, {start_year_predict}-2025)")

        # 5. Add remaining features and user-facing text
        df = self.add_features_for_prediction(df)

        return df.copy()

    # --- NEW PREDICTION FEATURE FUNCTION (STEP 3) ---

    def add_features_for_prediction(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Adds remaining features (non-time-sensitive) and user-facing fields.
        """
        df = df.copy()
        print(">>> 3. Engineering Remaining Features for Prediction...")

        # 1. Operational Flags
        df['covid_exposure'] = df['start_year'].between(2019, 2022).astype(int)

        # 2. Geography (SAFE: Only is_us)
        df_countries = self._safe_load('countries.txt', cols=['nct_id', 'name'])
        if not df_countries.empty:
            us_trials = df_countries[df_countries['name'] == 'United States']['nct_id'].unique()
            df['includes_us'] = df['nct_id'].isin(us_trials).astype(int)
        else:
            df['includes_us'] = 0

        # 3. Merge Standard Metadata
        df = self._merge_file(df, 'designs.txt', ['nct_id', 'allocation', 'intervention_model', 'masking', 'primary_purpose'])
        # NOTE: This line adds 'number_of_primary_outcomes_to_measure'
        df = self._merge_file(df, 'calculated_values.txt', ['nct_id', 'number_of_primary_outcomes_to_measure'])

        # 4. Sponsor Engineering
        df = self._engineer_sponsor_features(df)

        # 5. Complexity Engineering (Loads eligibility.txt and 'criteria' column)
        df = self._engineer_complexity(df)

        # 6. Agent Type
        df = self._engineer_agent_type(df)

        # 7. Smart Patterns (Rigor & Strictness)
        df = self._engineer_smart_patterns(df)

        # 8. Safe Features (DMC)
        df = self._engineer_safe_features(df)

        # 9. Text Features (Creates txt_tags, txt_criteria)
        df = self._prepare_text(df)

        # 10. ATTACH EMBEDDINGS (TEMPORARILY SKIPPED FOR COLLEAGUE'S WORKFLOW)
        df = self._attach_embeddings(df)

        # 11. Attach User-Facing Text Fields
        df = self._load_user_facing_text(df)

        # --- CLEANUP (FIXED LOGIC) ---
        old_col = 'number_of_primary_outcomes_to_measure'
        new_col = 'num_primary_endpoints'

        if old_col in df.columns:
            df.rename(columns={old_col: new_col}, inplace=True)
            # Convert to numeric and fill NaNs in the new column
            df[new_col] = pd.to_numeric(df[new_col], errors='coerce').fillna(1)
        else:
            # If the column was never loaded, create it with the default value 1
            df[new_col] = 1
        # -----------------------------

        return df

    # --- NEW HELPER FUNCTION FOR UI/USER-FACING DATA ---

    def _load_user_facing_text(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Loads text fields that are useful for the UI but not necessarily for the model.
        """
        print("    -> Loading User-Facing Text (Brief Summary, Detailed Description)...")

        # 1. Brief Summary
        cols_brief = ['nct_id', 'description']
        df_brief = self._safe_load('brief_summaries.txt', cols=cols_brief)
        if not df_brief.empty:
            df_brief = df_brief.rename(columns={'description': 'brief_summary_text'})
            df = df.merge(df_brief.drop_duplicates('nct_id'), on='nct_id', how='left')
        else:
            df['brief_summary_text'] = ""

        # 2. Detailed Description
        cols_detailed = ['nct_id', 'description']
        df_detailed = self._safe_load('detailed_descriptions.txt', cols=cols_detailed)
        if not df_detailed.empty:
            df_detailed = df_detailed.rename(columns={'description': 'detailed_description_text'})
            df = df.merge(df_detailed.drop_duplicates('nct_id'), on='nct_id', how='left')
        else:
            df['detailed_description_text'] = ""

        return df

    # --- ALL OTHER NECESSARY HELPER METHODS (KEPT FOR FUNCTIONALITY) ---

    def _engineer_smart_patterns(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Smart Patterns (Rigor & Strictness)...")
        def get_masking_score(val):
            val = str(val).lower()
            if 'quadruple' in val: return 3
            if 'double' in val: return 2
            if 'single' in val: return 1
            return 0
        def get_allocation_score(val):
            return 1 if 'randomized' in str(val).lower() else 0
        def get_model_score(val):
            val = str(val).lower()
            return 1 if 'crossover' in val or 'factorial' in val else 0
        df['score_masking'] = df['masking'].apply(get_masking_score)
        df['score_allocation'] = df['allocation'].apply(get_allocation_score)
        df['score_model'] = df['intervention_model'].apply(get_model_score)
        df['design_rigor_score'] = df['score_masking'] + df['score_allocation'] + df['score_model']
        df['is_gender_restricted'] = df['gender'].apply(lambda x: 0 if str(x).lower() == 'all' else 1)
        df['is_sick_only'] = df['healthy_volunteers'].apply(lambda x: 1 if str(x).lower() == 'no' else 0)
        for col in ['child', 'adult', 'older_adult']:
            if col in df.columns:
                df[col] = df[col].astype(str).apply(lambda x: 1 if x.lower() in ['true', '1', 'yes'] else 0)
            else:
                df[col] = 1
        df['eligibility_strictness_score'] = (
            df['is_gender_restricted'] +
            df['is_sick_only'] +
            (1 - df['child']) +
            (1 - df['older_adult'])
        )
        return df

    def _engineer_agent_type(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Agent Type (Bulletproof Classifier)...")
        if self.df_drugs.empty:
            df['agent_category'] = 'UNKNOWN'
            return df
        type_priority = {'GENETIC': 1,'BIOLOGICAL': 2,'DRUG': 3,'DIETARY SUPPLEMENT': 4,'OTHER': 5}
        df_int = self.df_drugs.copy()
        df_int['type_upper'] = df_int['intervention_type'].str.upper()
        df_int['priority'] = df_int['type_upper'].map(lambda x: type_priority.get(x, 5))
        df_int = df_int.sort_values('priority')
        best_types = df_int.drop_duplicates('nct_id')[['nct_id', 'type_upper', 'name']]
        df = df.merge(best_types, on='nct_id', how='left')
        def classify_molecule(row):
            itype = str(row['type_upper'])
            name = str(row['name']).lower()
            if 'placebo' in name: return 'PLACEBO_CTRL'
            if itype == 'GENETIC': return 'GENE_THERAPY'
            if any(x in name for x in ['car-t', 'chimeric antigen', 'autologous', 'allogeneic', 't-cell', 'nk cell']):
                return 'CELL_THERAPY'
            if name.endswith('cel'): return 'CELL_THERAPY'
            if any(x in name for x in ['crispr', 'cas9', 'mrna', 'sirna', 'antisense', 'oligonucleotide', 'plasmid', 'vector', 'aav']):
                return 'RNA_GENE_THERAPY'
            if itype == 'BIOLOGICAL': return 'BIOLOGIC'
            if 'mab' in name:
                if 'adc' in name or 'conjugate' in name: return 'ANTIBODY_DRUG_CONJUGATE'
                return 'MONOCLONAL_ANTIBODY'
            if name.endswith('cept'): return 'BIOLOGIC_FUSION'
            if 'vaccine' in name: return 'VACCINE'
            if any(x in name for x in ['interferon', 'interleukin', 'cytokine']): return 'IMMUNOTHERAPY'
            if name.endswith('ib') or 'ib ' in name:
                if 'tinib' in name: return 'KINASE_INHIBITOR_TYROSINE'
                if 'parib' in name: return 'PARP_INHIBITOR'
                if 'lisib' in name: return 'PI3K_INHIBITOR'
                return 'TARGETED_KINASE_INHIBITOR'
            if 'vastatin' in name: return 'STATIN_CHOLESTEROL'
            if name.endswith('stat') or 'stat ' in name: return 'ENZYME_INHIBITOR'
            if name.endswith('degib'): return 'HEDGEHOG_INHIBITOR'
            if name.endswith('clax'): return 'BCL2_INHIBITOR'
            chemo_stems = ['platin', 'taxel', 'rubicin', 'fluorouracil', 'gemcitabine',
                           'cyclophosphamide', 'methotrexate', 'etoposide', 'vincristine', 'vinblastine']
            if any(x in name for x in chemo_stems):
                return 'CHEMOTHERAPY'
            return 'SMALL_MOLECULE_OTHER'
        df['agent_category'] = df.apply(classify_molecule, axis=1)
        df.drop(columns=['type_upper', 'priority', 'name_y'], inplace=True, errors='ignore')
        if 'name_x' in df.columns: df.rename(columns={'name_x': 'official_title'}, inplace=True)
        return df

    def _engineer_safe_features(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Safe Protocol Features...")
        for col in ['has_dmc', 'is_fda_regulated_drug']:
            if col in df.columns:
                df[col] = df[col].astype(str).apply(
                    lambda x: 1 if x.lower() in ['true', 't', '1', 'yes'] else 0
                )
            else:
                df[col] = 0
        return df

    def _attach_medical_hierarchy(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Attaching Medical Hierarchy (Bridge: nct_id -> mesh_term -> area)...")
        cols_bridge = ['nct_id', 'mesh_term']
        df_bridge = self._safe_load('browse_conditions.txt', cols=cols_bridge)
        if df_bridge.empty:
            cols_cond = ['nct_id', 'name']
            df_bridge = self._safe_load('conditions.txt', cols=cols_cond)
            if not df_bridge.empty:
                df_bridge.rename(columns={'name': 'mesh_term'}, inplace=True)
        mesh_path = os.path.join(self.data_path, 'mesh_lookup.csv')
        df_dictionary = pd.DataFrame()
        if os.path.exists(mesh_path):
            try:
                df_dictionary = pd.read_csv(mesh_path, sep='|', on_bad_lines='skip')
                if 'mesh_term' in df_dictionary.columns and 'therapeutic_area' in df_dictionary.columns:
                    df_dictionary = df_dictionary[['mesh_term', 'therapeutic_area']].drop_duplicates()
                    df_dictionary.rename(columns={'therapeutic_area': 'lookup_area'}, inplace=True)
                else:
                    df_dictionary = pd.DataFrame()
            except Exception as e:
                print(f"       [!] Error reading mesh_lookup.csv: {e}")
        if not df_bridge.empty:
            if not df_dictionary.empty:
                df_full_mesh = df_bridge.merge(df_dictionary, on='mesh_term', how='left')
            else:
                df_full_mesh = df_bridge
                df_full_mesh['lookup_area'] = np.nan
            df_grouped = df_full_mesh.groupby('nct_id').agg({
                'mesh_term': 'first',
                'lookup_area': 'first'
            }).reset_index()
            df = df.merge(df_grouped, on='nct_id', how='left')
        else:
            df['mesh_term'] = np.nan
            df['lookup_area'] = np.nan
        smart_path = os.path.join(self.data_path, 'smart_pathology_lookup.csv')
        if os.path.exists(smart_path):
            try:
                df_smart = pd.read_csv(smart_path)
                if 'nct_id' in df_smart.columns:
                    df = df.merge(df_smart, on='nct_id', how='left')
            except:
                pass
        if 'best_pathology' not in df.columns: df['best_pathology'] = np.nan
        if 'therapeutic_area' not in df.columns: df['therapeutic_area'] = np.nan
        def get_final_category(row):
            if pd.notna(row.get('best_pathology')) and str(row.get('best_pathology')) != 'Unknown':
                return row['best_pathology']
            if pd.notna(row.get('lookup_area')) and str(row.get('lookup_area')) != 'Other/Unclassified':
                return row['lookup_area']
            if pd.notna(row.get('mesh_term')):
                return row['mesh_term']
            if pd.notna(row.get('therapeutic_area')) and str(row.get('therapeutic_area')) != 'Other':
                return row['therapeutic_area']
            return 'Unclassified Condition'
        df['therapeutic_subgroup_name'] = df.apply(get_final_category, axis=1)
        df.drop(columns=['mesh_term', 'lookup_area'], inplace=True, errors='ignore')
        return df

    def _engineer_sponsor_features(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Sponsor Tiers...")
        cols_needed = ['nct_id', 'lead_or_collaborator', 'name', 'agency_class']
        df_sponsors = self._safe_load('sponsors.txt', cols=cols_needed)
        if df_sponsors.empty:
            df['sponsor_tier'] = 'TIER_2_OTHER'
            df['sponsor_clean'] = 'UNKNOWN'
            df['agency_class'] = 'UNKNOWN'
            return df
        leads = df_sponsors[df_sponsors['lead_or_collaborator'].str.lower() == 'lead'][['nct_id', 'name', 'agency_class']]
        leads = leads.rename(columns={'name': 'lead_sponsor'})
        leads = leads.drop_duplicates('nct_id')
        df = df.merge(leads, on='nct_id', how='left')
        df['lead_sponsor'] = df['lead_sponsor'].fillna('UNKNOWN')
        df['agency_class'] = df['agency_class'].fillna('UNKNOWN')
        clean_col = df['lead_sponsor'].astype(str).str.lower().str.strip()
        legal_pattern = r'[.,]|\binc\b|\bltd\b|\bllc\b|\bcorp\b|\bgmbh\b|\bsa\b|\bplc\b'
        clean_col = clean_col.str.replace(legal_pattern, '', regex=True).str.strip()
        mappings = {
            'Pfizer': ['pfizer', 'wyeth', 'hospira'], 'GSK': ['glaxo', 'gsk', 'smithkline'],
            'Novartis': ['novartis', 'sandoz'], 'AstraZeneca': ['astrazeneca', 'medimmune'],
            'Merck': ['merck', 'msd'], 'Roche': ['roche', 'genentech', 'hoffmann'],
            'Sanofi': ['sanofi', 'aventis', 'genzyme'], 'J&J': ['johnson & johnson', 'janssen'],
            'Bayer': ['bayer', 'monsanto'], 'Boehringer': ['boehringer'],
            'BMS': ['bristol-myers', 'squibb', 'celgene'], 'Lilly': ['lilly'],
            'Abbott': ['abbott', 'abbvie'], 'Amgen': ['amgen'],
            'Takeda': ['takeda', 'shire'], 'Gilead': ['gilead'],
            'Novo Nordisk': ['novo nordisk'], 'NIH': ['national cancer institute', 'nci', 'national institutes of health', 'nih']
        }
        final_names = clean_col.copy()
        for std, keys in mappings.items():
            pattern = '|'.join(keys)
            mask = clean_col.str.contains(pattern, case=False, regex=True)
            final_names.loc[mask] = std
        df['sponsor_clean'] = final_names
        def get_tier(name):
            if name in mappings.keys(): return 'TIER_1_GIANT'
            return 'TIER_2_OTHER'
        df['sponsor_tier'] = df['sponsor_clean'].apply(get_tier)
        return df

    def _engineer_complexity(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Protocol Complexity (Calculating Age Flags)...")
        cols_needed = ['nct_id', 'criteria', 'gender', 'healthy_volunteers', 'minimum_age', 'maximum_age']
        df_elig = self._safe_load('eligibilities.txt', cols=cols_needed)
        if df_elig.empty:
            df['criteria_len_log'] = 0
            for c in ['gender', 'healthy_volunteers', 'adult', 'child', 'older_adult']: df[c] = 0
            return df
        df_elig = df_elig.drop_duplicates('nct_id')
        df = df.merge(df_elig, on='nct_id', how='left')
        df['criteria_len_log'] = np.log1p(df['criteria'].astype(str).str.len().fillna(0))
        df['healthy_volunteers'] = df['healthy_volunteers'].astype(str).str.lower().apply(
            lambda x: 'no' if x in ['f', 'false', '0', 'no', 'nan'] else 'yes'
        )
        df['gender'] = df['gender'].fillna('UNKNOWN')
        def parse_age_to_years(val, default_val):
            if pd.isna(val) or str(val).lower() in ['n/a', 'nan', '', 'none']: return default_val
            try:
                match = re.search(r'(\d+(\.\d+)?)', str(val))
                if not match: return default_val
                num = float(match.group(1))
                text = str(val).lower()
                if 'month' in text: num /= 12.0
                elif 'week' in text: num /= 52.0
                elif 'day' in text: num /= 365.0
                elif 'hour' in text: num /= 8760.0
                return num
            except:
                return default_val
        df['min_age_years'] = df['minimum_age'].apply(lambda x: parse_age_to_years(x, 0.0))
        df['max_age_years'] = df['maximum_age'].apply(lambda x: parse_age_to_years(x, 100.0))
        df['child'] = (df['min_age_years'] < 18).astype(int)
        df['adult'] = ((df['max_age_years'] >= 18) & (df['min_age_years'] < 65)).astype(int)
        df['older_adult'] = (df['max_age_years'] > 65).astype(int)
        df.drop(columns=['minimum_age', 'maximum_age', 'min_age_years', 'max_age_years'], inplace=True, errors='ignore')
        return df

    def _calculate_competition(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Calculating Competition...")
        try:
            req_cols = ['start_year', 'therapeutic_area', 'phase']
            if not all(col in df.columns for col in req_cols):
                df['competition_broad'] = 0
                df['competition_niche'] = 0
                return df
            grid = df.groupby(['start_year', 'therapeutic_area']).size().reset_index(name='count')
            lookup = dict(zip(zip(grid['start_year'], grid['therapeutic_area']), grid['count']))
            def get_comp(row):
                y, area = row['start_year'], row['therapeutic_area']
                return lookup.get((y, area), 0) + lookup.get((y-1, area), 0)
            df['competition_broad'] = df.apply(get_comp, axis=1)
            if 'therapeutic_subgroup_name' in df.columns:
                grid_niche = df.groupby(['start_year', 'therapeutic_subgroup_name']).size().reset_index(name='count')
                lookup_niche = dict(zip(zip(grid_niche['start_year'], grid_niche['therapeutic_subgroup_name']), grid_niche['count']))
                def get_niche(row):
                    y, sub = row['start_year'], row['therapeutic_subgroup_name']
                    return lookup_niche.get((y, sub), 0) + lookup_niche.get((y-1, sub), 0)
                df['competition_niche'] = df.apply(get_niche, axis=1)
            else:
                df['competition_niche'] = 0
        except:
            df['competition_broad'] = 0
            df['competition_niche'] = 0
        return df

    def _prepare_text(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Preparing Text Features...")
        df_keys = self._safe_load('keywords.txt', cols=['nct_id', 'name'])
        if not df_keys.empty:
            keys_grouped = df_keys.groupby('nct_id')['name'].apply(lambda x: " ".join(x.dropna().astype(str))).reset_index(name='txt_keywords')
            df = df.merge(keys_grouped, on='nct_id', how='left')
        else:
            df['txt_keywords'] = ""
        if not self.df_drugs.empty:
            int_names = self.df_drugs.groupby('nct_id')['name'].apply(lambda x: " ".join(x.dropna().astype(str))).reset_index(name='txt_int_names')
            df = df.merge(int_names, on='nct_id', how='left')
        else:
            df['txt_int_names'] = ""
        text_cols = ['official_title', 'txt_keywords', 'txt_int_names', 'criteria']
        for c in text_cols:
            if c in df.columns: df[c] = df[c].fillna("")
        df['txt_tags'] = (df['official_title'] + " " + df['txt_keywords'] + " " + df['txt_int_names'])
        if 'criteria' in df.columns:
            df['txt_criteria'] = df['criteria']
            df.drop(columns=['txt_keywords', 'txt_int_names'], inplace=True, errors='ignore')
        return df

    # NOTE: _attach_embeddings is now a placeholder function

    def _attach_embeddings(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Attaching Vectorized Text Embeddings...")
        emb_path = os.path.join(self.data_path, 'embeddings_with_nctid.csv')

        # 1. Check if file exists
        if not os.path.exists(emb_path):
            print("    [!] Warning: embeddings_with_nctid.csv not found.")
            return df

        # 2. LOAD AND MERGE
        try:
            # Load the embeddings CSV
            df_emb = pd.read_csv(emb_path)

            # Check if 'nct_id' is present to perform the merge
            if 'nct_id' in df_emb.columns:
                # Merge logic: Left join on nct_id
                df = df.merge(df_emb, on='nct_id', how='left')
                print(f"       -> Successfully merged embeddings (Columns added: {df_emb.shape[1] - 1})")
            else:
                print("       [!] Error: 'nct_id' column missing in embeddings file. Cannot merge.")

        except Exception as e:
            print(f"       [!] Error reading/merging embeddings: {e}")

        return df


    # NOTE: _attach_p_values is ONLY called in the original training pipeline (not in prediction)
    def _attach_p_values(self, df: pd.DataFrame) -> pd.DataFrame:
        # This function is not used in the prediction pipeline
        return df

    def _merge_file(self, df: pd.DataFrame, filename: str, cols: list, filter_col: Optional[str] = None, filter_val: Optional[str] = None) -> pd.DataFrame:
        try:
            aux = self._safe_load(filename, cols=cols + ([filter_col] if filter_col else []))
            if aux.empty: return df
            if filter_col:
                aux = aux[aux[filter_col] == filter_val].drop(columns=[filter_col])
            aux = aux.drop_duplicates('nct_id')
            return df.merge(aux, on='nct_id', how='left')
        except:
            return df

    def save(self, df: pd.DataFrame, filename: str = 'data_predict.csv'):
        out_path = os.path.join(self.data_path, filename)
        df.to_csv(out_path, index=False)
        print(f">>> Saved {len(df)} rows to {out_path}")

# ====================================================================
# 2. EXECUTION CODE (The part that runs the process)
# ====================================================================

try:
    # 1. Instantiate the loader
    loader = ClinicalTrialLoader(data_path=DATA_PATH)

    # --- STEP 1: Load and Calculate Time-Sensitive Features on the Full Cohort (2005-2025) ---
    df_full = loader.load_full_cohort_for_competition(min_year=2005, max_year=2025)

    # --- STEP 2: Generate Prediction Set (2024-2025, Active/Pending) ---
    # Note: start_year_predict=2024 enforces the new requirement
    df_predict = loader.load_and_clean_prediction_set(df_full, start_year_predict=2024)

    # 3. Save the final dataset for prediction
    loader.save(df_predict)

    print("\n[SUCCESS] Prediction cohort ready for model scoring and embedding generation.")

except ValueError as e:
    print(f"\n[CRITICAL FAILURE] Data loading stopped: {e}")
    print("ACTION REQUIRED: Check if 'studies.txt' is present in the DATA_PATH folder.")
except Exception as e:
    print(f"\n[UNEXPECTED FAILURE] An unexpected error occurred: {e}")
    print("ACTION REQUIRED: Check file paths, file names, and data integrity.")

DEBUG: Data Path is set to: /home/delaunan/code/delaunan/clintrialpredict/data
--- STARTING PREDICTION COHORT GENERATION ---
>>> 1. Loading Full Cohort (2005-2025) for Competition Calculation...
    -> Attaching Medical Hierarchy (Bridge: nct_id -> mesh_term -> area)...
    -> Calculating Competition...
    Full Cohort Size: 137724 trials.
>>> 2. Filtering for Prediction Set (2024-2025, Active/Pending Statuses)...
    Prediction Cohort: 13747 trials (Active/Pending Phase 2/3, 2024-2025)
>>> 3. Engineering Remaining Features for Prediction...
    -> Engineering Sponsor Tiers...
    -> Engineering Protocol Complexity (Calculating Age Flags)...
    -> Engineering Agent Type (Bulletproof Classifier)...
    -> Engineering Smart Patterns (Rigor & Strictness)...
    -> Engineering Safe Protocol Features...
    -> Preparing Text Features...
    -> Attaching Vectorized Text Embeddings...
       -> Successfully merged embeddings (Columns added: 20)
    -> Loading User-Facing Text (Brief Summary,

In [5]:
import pandas as pd
import numpy as np
import os

# ====================================================================
# CONFIGURATION
# ====================================================================
# This MUST match the path used to generate the files
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
TRAIN_FILE = 'project_data.csv'
PREDICT_FILE = 'data_predict.csv'
AUDIT_REPORT_FILE = 'data_audit_report.txt'

# ====================================================================
# 1. LOAD DATASETS
# ====================================================================
try:
    df_train = pd.read_csv(os.path.join(DATA_PATH, TRAIN_FILE))
    df_predict = pd.read_csv(os.path.join(DATA_PATH, PREDICT_FILE))

    # Ensure all columns are treated as strings for initial comparison
    train_cols = set(df_train.columns.astype(str))
    predict_cols = set(df_predict.columns.astype(str))

    report = []
    report.append("=" * 80)
    report.append(f"PREDICTION DATASET AUDIT: {PREDICT_FILE}")
    report.append("=" * 80)
    report.append(f"Training Set Size (project_data.csv): {len(df_train)} rows x {len(train_cols)} cols")
    report.append(f"Prediction Set Size (data_predict.csv): {len(df_predict)} rows x {len(predict_cols)} cols")
    report.append("-" * 80)

except FileNotFoundError as e:
    report.append(f"CRITICAL ERROR: File not found. Check DATA_PATH and filenames. Error: {e}")
    print("\n".join(report))
    sys.exit()
except Exception as e:
    report.append(f"CRITICAL ERROR: Failed to load CSVs. Error: {e}")
    print("\n".join(report))
    sys.exit()


# ====================================================================
# 2. COLUMN COMPARISON
# ====================================================================

# Identify column sets
common_cols = sorted(list(train_cols.intersection(predict_cols)))
new_cols = sorted(list(predict_cols.difference(train_cols)))
missing_cols = sorted(list(train_cols.difference(predict_cols)))

report.append("\n" + "=" * 30 + " COLUMN COMPARISON " + "=" * 30)
report.append(f"Total Common Columns: {len(common_cols)}")
report.append(f"Total NEW Columns in Prediction Set: {len(new_cols)}")
report.append(f"Total MISSING Columns from Training Set: {len(missing_cols)}")
report.append("-" * 80)

report.append("\n[A] NEW COLUMNS (User-Facing/NLP Fields)")
report.append("--------------------------------------------------------------------------------")
if new_cols:
    for col in new_cols:
        report.append(f"- {col}")
else:
    report.append("- NONE (Unexpected - should have UI/NLP fields)")

report.append("\n[B] MISSING COLUMNS (Expected Exclusions)")
report.append("--------------------------------------------------------------------------------")
if missing_cols:
    for col in missing_cols:
        # Flag expected leakage columns
        if col in ['target', 'min_p_value', 'scientific_success']:
            report.append(f"- {col} (EXPECTED: Target/Leakage Feature)")
        else:
            report.append(f"- {col} (WARNING: Missing from Prediction Set)")
else:
    report.append("- NONE (All training features are present or explicitly excluded)")


# ====================================================================
# 3. DATA QUALITY & COMPLETENESS (Prediction Set)
# ====================================================================

report.append("\n" + "=" * 30 + " DATA QUALITY & COMPLETENESS " + "=" * 24)

# Define critical features for production quality check
CRITICAL_FEATURES = [
    'design_rigor_score', 'agent_category', 'competition_broad',
    'sponsor_tier', 'num_primary_endpoints', 'criteria_len_log',
    'therapeutic_area', 'overall_status'
]

quality_data = []
total_rows = len(df_predict)
total_missing_cells = 0

for col in common_cols + new_cols:
    dtype = df_predict[col].dtype
    n_null = df_predict[col].isnull().sum()
    pct_null = (n_null / total_rows) * 100
    n_unique = df_predict[col].nunique()

    total_missing_cells += n_null

    is_critical = "CRITICAL" if col in CRITICAL_FEATURES else ""

    quality_data.append({
        'Feature': col,
        'Type': dtype,
        'Nulls (%)': f"{pct_null:.2f}%",
        'Unique': n_unique,
        'Flag': is_critical
    })

df_quality = pd.DataFrame(quality_data).sort_values(by='Nulls (%)', ascending=False)
report.append(df_quality.to_markdown(index=False))

# Overall Completeness Score
total_cells = total_rows * len(df_predict.columns)
completeness_score = 100 * (1 - (total_missing_cells / total_cells))
report.append("-" * 80)
report.append(f"OVERALL DATA COMPLETENESS SCORE: {completeness_score:.2f}%")
report.append("-" * 80)


# ====================================================================
# 4. STATISTICAL COMPARISON (DATA DRIFT CHECK)
# ====================================================================

report.append("\n" + "=" * 30 + " STATISTICAL DRIFT CHECK (Train vs. Predict) " + "=" * 13)

# Select key numerical features for comparison
NUMERIC_FEATURES = [
    'design_rigor_score', 'competition_broad', 'competition_niche',
    'num_primary_endpoints', 'number_of_arms', 'criteria_len_log'
]

drift_data = []
for col in NUMERIC_FEATURES:
    if col in common_cols:
        train_mean = df_train[col].mean()
        predict_mean = df_predict[col].mean()
        diff_pct = 100 * (predict_mean - train_mean) / train_mean

        drift_data.append({
            'Feature': col,
            'Train Mean': f"{train_mean:.2f}",
            'Predict Mean': f"{predict_mean:.2f}",
            'Difference (%)': f"{diff_pct:.2f}%"
        })

df_drift = pd.DataFrame(drift_data)
report.append(df_drift.to_markdown(index=False))

report.append("\n" + "=" * 30 + " CATEGORICAL DRIFT CHECK (Top 5 Categories) " + "=" * 11)

# Categorical comparison for the strongest features
CATEGORICAL_FEATURES = ['agent_category', 'therapeutic_area', 'phase', 'sponsor_tier']

for col in CATEGORICAL_FEATURES:
    if col in common_cols:
        train_counts = df_train[col].value_counts(normalize=True).head(5)
        predict_counts = df_predict[col].value_counts(normalize=True).head(5)

        combined = pd.concat([train_counts, predict_counts], axis=1, keys=['Train_Pct', 'Predict_Pct']).fillna(0)
        combined['Difference'] = combined['Predict_Pct'] - combined['Train_Pct']

        report.append(f"\n--- Feature: {col} ---")
        report.append(combined.sort_values(by='Difference', ascending=False).to_markdown())


# ====================================================================
# 5. FINAL OUTPUT
# ====================================================================

final_report = "\n".join(report)
print(final_report)

# Save the report to a file
with open(os.path.join(DATA_PATH, AUDIT_REPORT_FILE), 'w') as f:
    f.write(final_report)

print(f"\n[REPORT SAVED] Detailed audit report saved to: {os.path.join(DATA_PATH, AUDIT_REPORT_FILE)}")

PREDICTION DATASET AUDIT: data_predict.csv
Training Set Size (project_data.csv): 48294 rows x 147 cols
Prediction Set Size (data_predict.csv): 13747 rows x 69 cols
--------------------------------------------------------------------------------

============================== COLUMN COMPARISON ==============================
Total Common Columns: 64
Total NEW Columns in Prediction Set: 5
Total MISSING Columns from Training Set: 83
--------------------------------------------------------------------------------

[A] NEW COLUMNS (User-Facing/NLP Fields)
--------------------------------------------------------------------------------
- brief_summary_text
- brief_title
- criteria
- detailed_description_text
- official_title

[B] MISSING COLUMNS (Expected Exclusions)
--------------------------------------------------------------------------------
- emb_20 (WARNING: Missing from Prediction Set)
- emb_21 (WARNING: Missing from Prediction Set)
- emb_22 (WARNING: Missing from Prediction Set)
- e

In [2]:
import pandas as pd
import numpy as np
import os
import csv
import re
from typing import Optional

# ====================================================================
# 0. CONFIGURATION (THE ONLY LINE YOU MUST VERIFY)
# ====================================================================
# This MUST be the absolute path to the FOLDER containing your AACT .txt files.
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
NLP_OUTPUT_FILE = 'nlp_criteria_full_history_2005_2025.csv'
# --------------------------------------------------------------------

print(f"DEBUG: Data Path is set to: {DATA_PATH}")
print("--- STARTING FULL HISTORY NLP CRITERIA GENERATION ---")

# ====================================================================
# 1. CLINICAL TRIAL LOADER CLASS (Full Definition)
# ====================================================================

class ClinicalTrialLoader:
    """
    A specialized loader for generating the full historical NLP criteria dataset.
    """
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.df_drugs = pd.DataFrame()

        # --- STRATEGY A: PERFECT ---
        self.params_perfect = {
            "sep": "|", "dtype": str, "header": 0, "quotechar": '"',
            "quoting": csv.QUOTE_MINIMAL, "low_memory": False, "on_bad_lines": "warn"
        }

        # --- STRATEGY B: ROBUST ---
        self.params_robust = {
            "sep": "|", "dtype": str, "header": 0, "quotechar": '"',
            "quoting": 3, "low_memory": False, "on_bad_lines": "warn"
        }

    def _safe_load(self, filename: str, cols: Optional[list] = None) -> pd.DataFrame:
        full_path = os.path.join(self.data_path, filename)
        if not os.path.exists(full_path):
            print(f"   [!] Warning: File not found {filename}. Features will be empty.")
            return pd.DataFrame()

        try:
            return pd.read_csv(full_path, usecols=cols, **self.params_perfect)
        except Exception as e:
            print(f"   [!] Formatting error in {filename}. Switching to Robust Mode...")
            try:
                return pd.read_csv(full_path, usecols=cols, **self.params_robust)
            except Exception as e2:
                print(f"   [x] CRITICAL: Could not load {filename}. Error: {e2}")
                return pd.DataFrame()

    # --- MAIN FUNCTION: LOAD FULL HISTORY NLP CRITERIA ---

    def load_full_history_nlp_set(self, min_year: int = 2005, max_year: int = 2025) -> pd.DataFrame:
        """
        Loads the full historical dataset (2005-2025) for NLP embedding generation,
        including Phase 1 trials to maximize data for BioBERT training.
        """
        print(f">>> 1. Loading FULL HISTORY NLP Cohort ({min_year}-{max_year})...")

        # 1. Load Core Columns (Minimal set for filtering)
        cols_studies = ['nct_id', 'overall_status', 'study_type', 'phase', 'start_date']
        df = self._safe_load('studies.txt', cols=cols_studies)
        if df.empty:
            raise ValueError("Critical Error: 'studies.txt' failed to load.")

        # 2. Apply Core Filters (Interventional, Drug)
        df = df[df['study_type'].str.upper() == 'INTERVENTIONAL'].copy()

        cols_int = ['nct_id', 'intervention_type']
        df_int = self._safe_load('interventions.txt', cols=cols_int)
        if not df_int.empty:
            target_types = ['DRUG', 'BIOLOGICAL', 'GENETIC']
            drug_ids = df_int[df_int['intervention_type'].str.upper().isin(target_types)]['nct_id'].unique()
            df = df[df['nct_id'].isin(drug_ids)]

        # 3. CRITICAL CHANGE: Phase Filter (Keep Phase 1, Phase 2, Phase 3)
        # We only exclude EARLY_PHASE1, PHASE4, and NA
        excluded_phases = ['EARLY_PHASE1', 'PHASE4', 'NA']
        df = df[~df['phase'].astype(str).str.upper().isin(excluded_phases)]
        df = df.dropna(subset=['phase'])

        # 4. Apply Date Filter (2005-2025)
        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        df['start_year'] = df['start_date'].dt.year
        df = df[df['start_year'].between(min_year, max_year)].copy()

        # 5. CRITICAL: Load Eligibility Criteria Text
        cols_elig = ['nct_id', 'criteria']
        df_elig = self._safe_load('eligibilities.txt', cols=cols_elig)

        if df_elig.empty:
            print("    [!] Warning: 'eligibilities.txt' failed to load. Returning empty DataFrame.")
            return pd.DataFrame(columns=['nct_id', 'criteria'])

        # 6. Final Merge and Cleanup
        # Merge on nct_id to get the criteria for the filtered studies
        df_final = df[['nct_id']].merge(df_elig.drop_duplicates('nct_id'), on='nct_id', how='inner')


        # Rename column for clarity in the final output file
        df_final.rename(columns={'criteria': 'txt_criteria'}, inplace=True)

        print(f"    Final NLP Cohort Size: {len(df_final)} trials (Full History, Phase 1/2/3)")
        return df_final[['nct_id', 'txt_criteria']].copy()

    def save(self, df: pd.DataFrame, filename: str = 'project_data.csv'):
        out_path = os.path.join(self.data_path, filename)
        df.to_csv(out_path, index=False)
        print(f">>> Saved {len(df)} rows to {out_path}")

# ====================================================================
# 2. EXECUTION CODE
# ====================================================================

try:
    # 1. Instantiate the loader
    loader = ClinicalTrialLoader(data_path=DATA_PATH)

    # 2. Generate the NLP-focused dataset (2005-2025, including Phase 1)
    df_nlp_full = loader.load_full_history_nlp_set(min_year=2005, max_year=2025)

    # 3. Save the final dataset
    loader.save(df_nlp_full, filename=NLP_OUTPUT_FILE)

    print(f"\n[SUCCESS] Full History NLP criteria set ready for colleague at: {os.path.join(DATA_PATH, NLP_OUTPUT_FILE)}")
    print("WARNING: This file is large and contains the entire 2005-2025 Phase 1/2/3 cohort. BioBERT processing will take significant time/resources.")

except ValueError as e:
    print(f"\n[CRITICAL FAILURE] Data loading stopped: {e}")
    print("ACTION REQUIRED: Check if 'studies.txt' is present in the DATA_PATH folder.")
except Exception as e:
    print(f"\n[UNEXPECTED FAILURE] An unexpected error occurred: {e}")
    print("ACTION REQUIRED: Check file paths, file names, and data integrity.")

DEBUG: Data Path is set to: /home/delaunan/code/delaunan/clintrialpredict/data
--- STARTING FULL HISTORY NLP CRITERIA GENERATION ---
>>> 1. Loading FULL HISTORY NLP Cohort (2005-2025)...
    Final NLP Cohort Size: 137724 trials (Full History, Phase 1/2/3)
>>> Saved 137724 rows to /home/delaunan/code/delaunan/clintrialpredict/data/nlp_criteria_full_history_2005_2025.csv

[SUCCESS] Full History NLP criteria set ready for colleague at: /home/delaunan/code/delaunan/clintrialpredict/data/nlp_criteria_full_history_2005_2025.csv


In [2]:
import pandas as pd
import numpy as np
import os
import csv
import re
import sys
from typing import Optional

# ====================================================================
# 0. CONFIGURATION
# ====================================================================
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
INPUT_LIST_FILE = 'data_predict.csv'
OUTPUT_FILE = 'demo_set.csv'

print(f"DEBUG: Data Path is set to: {DATA_PATH}")
print("--- STARTING DEMO SET GENERATION ---")

# ====================================================================
# 1. CLINICAL TRIAL LOADER CLASS
# ====================================================================

class ClinicalTrialLoader:
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.df_drugs = pd.DataFrame()
        self.params_perfect = {"sep": "|", "dtype": str, "header": 0, "quotechar": '"', "quoting": csv.QUOTE_MINIMAL, "low_memory": False, "on_bad_lines": "warn"}
        self.params_robust = {"sep": "|", "dtype": str, "header": 0, "quotechar": '"', "quoting": 3, "low_memory": False, "on_bad_lines": "warn"}

    def _safe_load(self, filename: str, cols: Optional[list] = None) -> pd.DataFrame:
        full_path = os.path.join(self.data_path, filename)
        if not os.path.exists(full_path): return pd.DataFrame()
        try: return pd.read_csv(full_path, usecols=cols, **self.params_perfect)
        except:
            try: return pd.read_csv(full_path, usecols=cols, **self.params_robust)
            except: return pd.DataFrame()

    def load_specific_trials(self, nct_ids: list) -> pd.DataFrame:
        print(f">>> 1. Loading Specific Cohort ({len(nct_ids)} trials)...")
        cols_studies = ['nct_id', 'overall_status', 'study_type', 'phase', 'start_date', 'number_of_arms', 'official_title', 'why_stopped', 'has_dmc', 'is_fda_regulated_drug', 'brief_title']
        df = self._safe_load('studies.txt', cols=cols_studies)

        # FILTER
        df = df[df['nct_id'].isin(nct_ids)].copy()

        # Load Interventions
        cols_int = ['nct_id', 'intervention_type', 'name']
        df_int = self._safe_load('interventions.txt', cols=cols_int)
        if not df_int.empty:
            df_int = df_int[df_int['nct_id'].isin(nct_ids)]
            target_types = ['DRUG', 'BIOLOGICAL', 'GENETIC']
            self.df_drugs = df_int[df_int['intervention_type'].str.upper().isin(target_types + ['DIETARY SUPPLEMENT', 'OTHER'])].copy()

        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        df['start_year'] = df['start_date'].dt.year

        df = self._attach_medical_hierarchy(df)
        df = self._calculate_competition(df)
        print(f"    Base Cohort Loaded: {len(df)} trials.")
        return df

    def add_features_for_prediction(self, df: pd.DataFrame) -> pd.DataFrame:
        print(">>> 2. Engineering Features...")
        df['covid_exposure'] = df['start_year'].between(2019, 2022).astype(int)

        df_countries = self._safe_load('countries.txt', cols=['nct_id', 'name'])
        if not df_countries.empty:
            us_trials = df_countries[df_countries['name'] == 'United States']['nct_id'].unique()
            df['includes_us'] = df['nct_id'].isin(us_trials).astype(int)
        else: df['includes_us'] = 0

        df = self._merge_file(df, 'designs.txt', ['nct_id', 'allocation', 'intervention_model', 'masking', 'primary_purpose'])
        df = self._merge_file(df, 'calculated_values.txt', ['nct_id', 'number_of_primary_outcomes_to_measure'])

        df = self._engineer_sponsor_features(df)
        df = self._engineer_complexity(df) # <--- This loads gender/healthy_volunteers
        df = self._engineer_agent_type(df)
        df = self._engineer_smart_patterns(df) # <--- This uses gender
        df = self._engineer_safe_features(df)
        df = self._prepare_text(df)
        df = self._attach_embeddings(df)
        df = self._load_user_facing_text(df)

        old_col, new_col = 'number_of_primary_outcomes_to_measure', 'num_primary_endpoints'
        if old_col in df.columns:
            df.rename(columns={old_col: new_col}, inplace=True)
            df[new_col] = pd.to_numeric(df[new_col], errors='coerce').fillna(1)
        else: df[new_col] = 1
        return df

    def _engineer_complexity(self, df):
        # FIX: Actually load the file to get 'gender'
        cols_needed = ['nct_id', 'criteria', 'gender', 'healthy_volunteers', 'minimum_age', 'maximum_age']
        df_elig = self._safe_load('eligibilities.txt', cols=cols_needed)

        if not df_elig.empty:
            df_elig = df_elig.drop_duplicates('nct_id')
            df = df.merge(df_elig, on='nct_id', how='left')

        # Fill missing values if merge failed for some rows
        if 'gender' not in df.columns: df['gender'] = 'ALL'
        if 'healthy_volunteers' not in df.columns: df['healthy_volunteers'] = 'No'
        if 'criteria' not in df.columns: df['criteria'] = ''

        df['criteria_len_log'] = np.log1p(df['criteria'].astype(str).str.len().fillna(0))
        df['healthy_volunteers'] = df['healthy_volunteers'].astype(str).str.lower().apply(lambda x: 'no' if x in ['f', 'false', '0', 'no'] else 'yes')

        # Simple Age Logic (Placeholder)
        df['child'] = 0
        df['adult'] = 1
        df['older_adult'] = 0
        return df

    def _engineer_smart_patterns(self, df):
        def get_masking_score(val):
            val = str(val).lower()
            if 'quadruple' in val: return 3
            if 'double' in val: return 2
            if 'single' in val: return 1
            return 0
        df['score_masking'] = df['masking'].apply(get_masking_score)
        df['score_allocation'] = df['allocation'].apply(lambda x: 1 if 'randomized' in str(x).lower() else 0)
        df['score_model'] = df['intervention_model'].apply(lambda x: 1 if 'crossover' in str(x).lower() or 'factorial' in str(x).lower() else 0)
        df['design_rigor_score'] = df['score_masking'] + df['score_allocation'] + df['score_model']

        # Now 'gender' is guaranteed to exist
        df['gender'] = df['gender'].fillna('ALL')
        df['is_gender_restricted'] = df['gender'].apply(lambda x: 0 if str(x).lower() == 'all' else 1)

        df['is_sick_only'] = df['healthy_volunteers'].apply(lambda x: 1 if str(x).lower() == 'no' else 0)

        for c in ['child', 'adult', 'older_adult']:
            if c not in df.columns: df[c] = 1

        df['eligibility_strictness_score'] = df['is_gender_restricted'] + df['is_sick_only'] + (1 - df['child']) + (1 - df['older_adult'])
        return df

    def _engineer_agent_type(self, df):
        if self.df_drugs.empty:
            df['agent_category'] = 'UNKNOWN'; return df

        type_priority = {'GENETIC': 1,'BIOLOGICAL': 2,'DRUG': 3,'DIETARY SUPPLEMENT': 4,'OTHER': 5}
        df_int = self.df_drugs.copy()
        df_int['type_upper'] = df_int['intervention_type'].str.upper()
        df_int['priority'] = df_int['type_upper'].map(lambda x: type_priority.get(x, 5))
        df_int = df_int.sort_values('priority')
        best_types = df_int.drop_duplicates('nct_id')[['nct_id', 'type_upper', 'name']]
        df = df.merge(best_types, on='nct_id', how='left')

        def classify_molecule(row):
            name = str(row['name']).lower()
            itype = str(row['type_upper'])
            if 'placebo' in name: return 'PLACEBO_CTRL'
            if itype == 'GENETIC' or any(x in name for x in ['car-t', 'crispr', 'mrna']): return 'GENE_THERAPY'
            if itype == 'BIOLOGICAL' or 'mab' in name: return 'BIOLOGIC'
            return 'SMALL_MOLECULE_OTHER'

        df['agent_category'] = df.apply(classify_molecule, axis=1)
        df.drop(columns=['type_upper', 'priority', 'name_y'], inplace=True, errors='ignore')
        if 'name_x' in df.columns: df.rename(columns={'name_x': 'official_title'}, inplace=True)
        return df

    def _engineer_safe_features(self, df):
        for c in ['has_dmc', 'is_fda_regulated_drug']:
            if c in df.columns: df[c] = df[c].astype(str).apply(lambda x: 1 if x.lower() in ['true', 't', '1', 'yes'] else 0)
            else: df[c] = 0
        return df

    def _attach_medical_hierarchy(self, df):
        df['therapeutic_subgroup_name'] = 'Unknown'
        df['therapeutic_area'] = 'Other'
        return df

    def _engineer_sponsor_features(self, df):
        cols_needed = ['nct_id', 'lead_or_collaborator', 'name', 'agency_class']
        df_sponsors = self._safe_load('sponsors.txt', cols=cols_needed)
        if df_sponsors.empty:
            df['sponsor_tier'] = 'TIER_2_OTHER'; df['sponsor_clean'] = 'UNKNOWN'; return df

        leads = df_sponsors[df_sponsors['lead_or_collaborator'].str.lower() == 'lead'].drop_duplicates('nct_id')
        leads = leads.rename(columns={'name': 'lead_sponsor'})
        df = df.merge(leads[['nct_id', 'lead_sponsor', 'agency_class']], on='nct_id', how='left')

        big_pharma = ['pfizer', 'gsk', 'novartis', 'astrazeneca', 'merck', 'roche', 'sanofi', 'johnson', 'bayer', 'lilly', 'abbvie', 'amgen']
        df['sponsor_clean'] = df['lead_sponsor'].astype(str).str.lower()
        df['sponsor_tier'] = df['sponsor_clean'].apply(lambda x: 'TIER_1_GIANT' if any(k in str(x) for k in big_pharma) else 'TIER_2_OTHER')
        return df

    def _calculate_competition(self, df):
        df['competition_broad'] = 0
        df['competition_niche'] = 0
        return df

    def _prepare_text(self, df):
        df['txt_criteria'] = df.get('criteria', '').fillna('')
        return df

    def _attach_embeddings(self, df):
        emb_path = os.path.join(self.data_path, 'embeddings_with_nctid.csv')
        if os.path.exists(emb_path):
            try:
                df_emb = pd.read_csv(emb_path)
                if 'nct_id' in df_emb.columns: df = df.merge(df_emb, on='nct_id', how='left')
            except: pass
        return df

    def _load_user_facing_text(self, df):
        df_brief = self._safe_load('brief_summaries.txt', cols=['nct_id', 'description'])
        if not df_brief.empty:
            df_brief = df_brief.rename(columns={'description': 'brief_summary_text'})
            df = df.merge(df_brief.drop_duplicates('nct_id'), on='nct_id', how='left')

        df_detailed = self._safe_load('detailed_descriptions.txt', cols=['nct_id', 'description'])
        if not df_detailed.empty:
            df_detailed = df_detailed.rename(columns={'description': 'detailed_description_text'})
            df = df.merge(df_detailed.drop_duplicates('nct_id'), on='nct_id', how='left')
        return df

    def _merge_file(self, df, filename, cols):
        aux = self._safe_load(filename, cols=cols).drop_duplicates('nct_id')
        if not aux.empty: return df.merge(aux, on='nct_id', how='left')
        return df

    def save(self, df, filename):
        out_path = os.path.join(self.data_path, filename)
        df.to_csv(out_path, index=False)
        print(f">>> Saved {len(df)} rows to {out_path}")

# ====================================================================
# 2. EXECUTION
# ====================================================================

try:
    input_df = pd.read_csv(os.path.join(DATA_PATH, INPUT_LIST_FILE))
    target_nct_ids = input_df['nct_id'].unique().tolist()
    print(f"Found {len(target_nct_ids)} unique NCT IDs to process.")
except FileNotFoundError:
    print(f"Error: Could not find {INPUT_LIST_FILE}. Please run the previous selection script first.")
    sys.exit()

loader = ClinicalTrialLoader(data_path=DATA_PATH)
df_demo = loader.load_specific_trials(target_nct_ids)
df_demo = loader.add_features_for_prediction(df_demo)
loader.save(df_demo, filename=OUTPUT_FILE)

print("\n[SUCCESS] Demo Set Generated.")
print(f"File: {os.path.join(DATA_PATH, OUTPUT_FILE)}")

DEBUG: Data Path is set to: /home/delaunan/code/delaunan/clintrialpredict/data
--- STARTING DEMO SET GENERATION ---
Found 452 unique NCT IDs to process.
>>> 1. Loading Specific Cohort (452 trials)...
    Base Cohort Loaded: 452 trials.
>>> 2. Engineering Features...
>>> Saved 452 rows to /home/delaunan/code/delaunan/clintrialpredict/data/demo_set.csv

[SUCCESS] Demo Set Generated.
File: /home/delaunan/code/delaunan/clintrialpredict/data/demo_set.csv


In [5]:
import pandas as pd
import numpy as np
import os
import csv
import re
import sys
from typing import Optional

# ====================================================================
# 0. CONFIGURATION
# ====================================================================
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
INPUT_LIST_FILE = 'data_predict.csv'
OUTPUT_FILE = 'demo_set.csv'

print(f"DEBUG: Data Path is set to: {DATA_PATH}")
print("--- STARTING DEMO SET GENERATION (FIXED) ---")

# ====================================================================
# 1. CLINICAL TRIAL LOADER CLASS
# ====================================================================

class ClinicalTrialLoader:
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.df_drugs = pd.DataFrame()
        self.params_perfect = {"sep": "|", "dtype": str, "header": 0, "quotechar": '"', "quoting": csv.QUOTE_MINIMAL, "low_memory": False, "on_bad_lines": "warn"}
        self.params_robust = {"sep": "|", "dtype": str, "header": 0, "quotechar": '"', "quoting": 3, "low_memory": False, "on_bad_lines": "warn"}

    def _safe_load(self, filename: str, cols: Optional[list] = None) -> pd.DataFrame:
        full_path = os.path.join(self.data_path, filename)
        if not os.path.exists(full_path): return pd.DataFrame()
        try: return pd.read_csv(full_path, usecols=cols, **self.params_perfect)
        except:
            try: return pd.read_csv(full_path, usecols=cols, **self.params_robust)
            except: return pd.DataFrame()

    def load_specific_trials(self, nct_ids: list) -> pd.DataFrame:
        print(f">>> 1. Loading Specific Cohort ({len(nct_ids)} trials)...")
        cols_studies = ['nct_id', 'overall_status', 'study_type', 'phase', 'start_date', 'number_of_arms', 'official_title', 'why_stopped', 'has_dmc', 'is_fda_regulated_drug', 'brief_title']
        df = self._safe_load('studies.txt', cols=cols_studies)

        # FILTER
        df = df[df['nct_id'].isin(nct_ids)].copy()

        # Load Interventions
        cols_int = ['nct_id', 'intervention_type', 'name']
        df_int = self._safe_load('interventions.txt', cols=cols_int)
        if not df_int.empty:
            df_int = df_int[df_int['nct_id'].isin(nct_ids)]
            target_types = ['DRUG', 'BIOLOGICAL', 'GENETIC']
            self.df_drugs = df_int[df_int['intervention_type'].str.upper().isin(target_types + ['DIETARY SUPPLEMENT', 'OTHER'])].copy()

        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        df['start_year'] = df['start_date'].dt.year

        df = self._attach_medical_hierarchy(df) # <--- FIX IS HERE
        df = self._calculate_competition(df)
        print(f"    Base Cohort Loaded: {len(df)} trials.")
        return df

    def add_features_for_prediction(self, df: pd.DataFrame) -> pd.DataFrame:
        print(">>> 2. Engineering Features...")
        df['covid_exposure'] = df['start_year'].between(2019, 2022).astype(int)

        df_countries = self._safe_load('countries.txt', cols=['nct_id', 'name'])
        if not df_countries.empty:
            us_trials = df_countries[df_countries['name'] == 'United States']['nct_id'].unique()
            df['includes_us'] = df['nct_id'].isin(us_trials).astype(int)
        else: df['includes_us'] = 0

        df = self._merge_file(df, 'designs.txt', ['nct_id', 'allocation', 'intervention_model', 'masking', 'primary_purpose'])
        df = self._merge_file(df, 'calculated_values.txt', ['nct_id', 'number_of_primary_outcomes_to_measure'])

        df = self._engineer_sponsor_features(df)
        df = self._engineer_complexity(df)
        df = self._engineer_agent_type(df)
        df = self._engineer_smart_patterns(df)
        df = self._engineer_safe_features(df)
        df = self._prepare_text(df)
        df = self._attach_embeddings(df)
        df = self._load_user_facing_text(df)

        old_col, new_col = 'number_of_primary_outcomes_to_measure', 'num_primary_endpoints'
        if old_col in df.columns:
            df.rename(columns={old_col: new_col}, inplace=True)
            df[new_col] = pd.to_numeric(df[new_col], errors='coerce').fillna(1)
        else: df[new_col] = 1
        return df

    def _attach_medical_hierarchy(self, df):
        # 1. Load Bridge (NCT -> Mesh Term)
        cols_bridge = ['nct_id', 'mesh_term']
        df_bridge = self._safe_load('browse_conditions.txt', cols=cols_bridge)

        # 2. Load Dictionary (Mesh Term -> Area)
        mesh_path = os.path.join(self.data_path, 'mesh_lookup.csv')
        df_dictionary = pd.DataFrame()
        if os.path.exists(mesh_path):
            try:
                df_dictionary = pd.read_csv(mesh_path, sep='|', on_bad_lines='skip')
            except: pass

        # 3. Merge
        if not df_bridge.empty:
            if not df_dictionary.empty:
                df_full = df_bridge.merge(df_dictionary, on='mesh_term', how='left')
            else:
                df_full = df_bridge
                df_full['therapeutic_area'] = 'Other'

            # Group by NCT to get one row per trial
            df_grouped = df_full.groupby('nct_id').first().reset_index()
            df = df.merge(df_grouped, on='nct_id', how='left')

        # 4. CRITICAL FIX: Ensure 'best_pathology' exists
        # In your training logic, best_pathology comes from smart_pathology_lookup.csv OR falls back to mesh_term
        smart_path = os.path.join(self.data_path, 'smart_pathology_lookup.csv')
        if os.path.exists(smart_path):
            try:
                df_smart = pd.read_csv(smart_path)
                if 'nct_id' in df_smart.columns:
                    df = df.merge(df_smart[['nct_id', 'best_pathology']], on='nct_id', how='left')
            except: pass

        # Fallbacks
        if 'best_pathology' not in df.columns:
            # If smart lookup failed, use mesh_term as proxy for best_pathology
            df['best_pathology'] = df.get('mesh_term', 'Unknown')

        df['best_pathology'] = df['best_pathology'].fillna('Unknown')
        df['therapeutic_area'] = df.get('therapeutic_area', 'Other').fillna('Other')
        df['therapeutic_subgroup_name'] = df.get('mesh_term', 'Unknown').fillna('Unknown')

        return df

    def _engineer_complexity(self, df):
        cols_needed = ['nct_id', 'criteria', 'gender', 'healthy_volunteers', 'minimum_age', 'maximum_age']
        df_elig = self._safe_load('eligibilities.txt', cols=cols_needed)

        if not df_elig.empty:
            df_elig = df_elig.drop_duplicates('nct_id')
            df = df.merge(df_elig, on='nct_id', how='left')

        if 'gender' not in df.columns: df['gender'] = 'ALL'
        if 'healthy_volunteers' not in df.columns: df['healthy_volunteers'] = 'No'
        if 'criteria' not in df.columns: df['criteria'] = ''

        df['criteria_len_log'] = np.log1p(df['criteria'].astype(str).str.len().fillna(0))
        df['healthy_volunteers'] = df['healthy_volunteers'].astype(str).str.lower().apply(lambda x: 'no' if x in ['f', 'false', '0', 'no'] else 'yes')

        df['child'] = 0
        df['adult'] = 1
        df['older_adult'] = 0
        return df

    def _engineer_smart_patterns(self, df):
        def get_masking_score(val):
            val = str(val).lower()
            if 'quadruple' in val: return 3
            if 'double' in val: return 2
            if 'single' in val: return 1
            return 0
        df['score_masking'] = df['masking'].apply(get_masking_score)
        df['score_allocation'] = df['allocation'].apply(lambda x: 1 if 'randomized' in str(x).lower() else 0)
        df['score_model'] = df['intervention_model'].apply(lambda x: 1 if 'crossover' in str(x).lower() or 'factorial' in str(x).lower() else 0)
        df['design_rigor_score'] = df['score_masking'] + df['score_allocation'] + df['score_model']

        df['gender'] = df['gender'].fillna('ALL')
        df['is_gender_restricted'] = df['gender'].apply(lambda x: 0 if str(x).lower() == 'all' else 1)
        df['is_sick_only'] = df['healthy_volunteers'].apply(lambda x: 1 if str(x).lower() == 'no' else 0)

        for c in ['child', 'adult', 'older_adult']:
            if c not in df.columns: df[c] = 1

        df['eligibility_strictness_score'] = df['is_gender_restricted'] + df['is_sick_only'] + (1 - df['child']) + (1 - df['older_adult'])
        return df

    def _engineer_agent_type(self, df):
        if self.df_drugs.empty:
            df['agent_category'] = 'UNKNOWN'; return df

        type_priority = {'GENETIC': 1,'BIOLOGICAL': 2,'DRUG': 3,'DIETARY SUPPLEMENT': 4,'OTHER': 5}
        df_int = self.df_drugs.copy()
        df_int['type_upper'] = df_int['intervention_type'].str.upper()
        df_int['priority'] = df_int['type_upper'].map(lambda x: type_priority.get(x, 5))
        df_int = df_int.sort_values('priority')
        best_types = df_int.drop_duplicates('nct_id')[['nct_id', 'type_upper', 'name']]
        df = df.merge(best_types, on='nct_id', how='left')

        def classify_molecule(row):
            name = str(row['name']).lower()
            itype = str(row['type_upper'])
            if 'placebo' in name: return 'PLACEBO_CTRL'
            if itype == 'GENETIC' or any(x in name for x in ['car-t', 'crispr', 'mrna']): return 'GENE_THERAPY'
            if itype == 'BIOLOGICAL' or 'mab' in name: return 'BIOLOGIC'
            return 'SMALL_MOLECULE_OTHER'

        df['agent_category'] = df.apply(classify_molecule, axis=1)
        df.drop(columns=['type_upper', 'priority', 'name_y'], inplace=True, errors='ignore')
        if 'name_x' in df.columns: df.rename(columns={'name_x': 'official_title'}, inplace=True)
        return df

    def _engineer_safe_features(self, df):
        for c in ['has_dmc', 'is_fda_regulated_drug']:
            if c in df.columns: df[c] = df[c].astype(str).apply(lambda x: 1 if x.lower() in ['true', 't', '1', 'yes'] else 0)
            else: df[c] = 0
        return df

    def _engineer_sponsor_features(self, df):
        cols_needed = ['nct_id', 'lead_or_collaborator', 'name', 'agency_class']
        df_sponsors = self._safe_load('sponsors.txt', cols=cols_needed)
        if df_sponsors.empty:
            df['sponsor_tier'] = 'TIER_2_OTHER'; df['sponsor_clean'] = 'UNKNOWN'; return df

        leads = df_sponsors[df_sponsors['lead_or_collaborator'].str.lower() == 'lead'].drop_duplicates('nct_id')
        leads = leads.rename(columns={'name': 'lead_sponsor'})
        df = df.merge(leads[['nct_id', 'lead_sponsor', 'agency_class']], on='nct_id', how='left')

        big_pharma = ['pfizer', 'gsk', 'novartis', 'astrazeneca', 'merck', 'roche', 'sanofi', 'johnson', 'bayer', 'lilly', 'abbvie', 'amgen']
        df['sponsor_clean'] = df['lead_sponsor'].astype(str).str.lower()
        df['sponsor_tier'] = df['sponsor_clean'].apply(lambda x: 'TIER_1_GIANT' if any(k in str(x) for k in big_pharma) else 'TIER_2_OTHER')
        return df

    def _calculate_competition(self, df):
        df['competition_broad'] = 0
        df['competition_niche'] = 0
        return df

    def _prepare_text(self, df):
        df['txt_criteria'] = df.get('criteria', '').fillna('')
        return df

    def _attach_embeddings(self, df):
        emb_path = os.path.join(self.data_path, 'embeddings_with_nctid.csv')
        if os.path.exists(emb_path):
            try:
                df_emb = pd.read_csv(emb_path)
                if 'nct_id' in df_emb.columns: df = df.merge(df_emb, on='nct_id', how='left')
            except: pass
        return df

    def _load_user_facing_text(self, df):
        df_brief = self._safe_load('brief_summaries.txt', cols=['nct_id', 'description'])
        if not df_brief.empty:
            df_brief = df_brief.rename(columns={'description': 'brief_summary_text'})
            df = df.merge(df_brief.drop_duplicates('nct_id'), on='nct_id', how='left')

        df_detailed = self._safe_load('detailed_descriptions.txt', cols=['nct_id', 'description'])
        if not df_detailed.empty:
            df_detailed = df_detailed.rename(columns={'description': 'detailed_description_text'})
            df = df.merge(df_detailed.drop_duplicates('nct_id'), on='nct_id', how='left')
        return df

    def _merge_file(self, df, filename, cols):
        aux = self._safe_load(filename, cols=cols).drop_duplicates('nct_id')
        if not aux.empty: return df.merge(aux, on='nct_id', how='left')
        return df

    def save(self, df, filename):
        out_path = os.path.join(self.data_path, filename)
        df.to_csv(out_path, index=False)
        print(f">>> Saved {len(df)} rows to {out_path}")

# ====================================================================
# 2. EXECUTION
# ====================================================================

try:
    input_df = pd.read_csv(os.path.join(DATA_PATH, INPUT_LIST_FILE))
    target_nct_ids = input_df['nct_id'].unique().tolist()
    print(f"Found {len(target_nct_ids)} unique NCT IDs to process.")
except FileNotFoundError:
    print(f"Error: Could not find {INPUT_LIST_FILE}. Please run the previous selection script first.")
    sys.exit()

loader = ClinicalTrialLoader(data_path=DATA_PATH)
df_demo = loader.load_specific_trials(target_nct_ids)
df_demo = loader.add_features_for_prediction(df_demo)
loader.save(df_demo, filename=OUTPUT_FILE)

print("\n[SUCCESS] Demo Set Generated.")
print(f"File: {os.path.join(DATA_PATH, OUTPUT_FILE)}")

DEBUG: Data Path is set to: /home/delaunan/code/delaunan/clintrialpredict/data
--- STARTING DEMO SET GENERATION (FIXED) ---
Found 452 unique NCT IDs to process.
>>> 1. Loading Specific Cohort (452 trials)...
    Base Cohort Loaded: 452 trials.
>>> 2. Engineering Features...
>>> Saved 452 rows to /home/delaunan/code/delaunan/clintrialpredict/data/demo_set.csv

[SUCCESS] Demo Set Generated.
File: /home/delaunan/code/delaunan/clintrialpredict/data/demo_set.csv


In [19]:
import pandas as pd
import numpy as np
import shap
import joblib
import os

# ====================================================================
# 0. CONFIGURATION
# ====================================================================
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
INPUT_FILE = 'demo_set.csv'
MODEL_PATH = '/home/delaunan/code/delaunan/clintrialpredict/models/ctp_model.joblib'

# ====================================================================
# 1. LOGIC DEFINITIONS (The "Brain")
# ====================================================================

def get_pillar(feature_name):
    """Maps raw features to Business Pillars"""
    n = str(feature_name).lower()
    if any(x in n for x in ['emb_', 'criteria', 'eligibility', 'gender', 'age', 'child', 'sick', 'healthy', 'adult']):
        return "Patient & Criteria"
    if any(x in n for x in ['phase', 'masking', 'arms', 'endpoints', 'dmc', 'rigor', 'allocation', 'model']):
        return "Protocol Design"
    if any(x in n for x in ['sponsor', 'agency', 'us', 'year', 'covid', 'location']):
        return "Sponsor & Operations"
    return "Therapeutic Landscape"

def tag_real_reason(text):
    """Tags the human text into categories"""
    if pd.isna(text) or str(text).strip() == '': return "Unknown"
    t = str(text).lower()

    # 1. EFFICACY (Matches Therapeutic Landscape)
    if any(x in t for x in ['efficacy', 'futility', 'benefit', 'effective', 'response', 'endpoint', 'inferior']):
        return "Therapeutic Landscape"
    # 2. SAFETY (Matches Protocol or Therapeutic)
    if any(x in t for x in ['safety', 'toxic', 'adverse', 'death', 'risk', 'mortality']):
        return "Protocol Design"
    # 3. ENROLLMENT (Matches Patient & Criteria)
    if any(x in t for x in ['enroll', 'recruit', 'accrual', 'participant', 'subject', 'sample']):
        return "Patient & Criteria"
    # 4. BUSINESS (Matches Sponsor)
    if any(x in t for x in ['business', 'fund', 'financ', 'sponsor', 'priorit', 'strategic', 'decision']):
        return "Sponsor & Operations"

    return "Other"

# ====================================================================
# 2. PROCESSING
# ====================================================================
print(">>> Loading Data & Model...")
df = pd.read_csv(os.path.join(DATA_PATH, INPUT_FILE))
model = joblib.load(MODEL_PATH)

# Prepare Features
# (We assume demo_set.csv has all columns needed. We drop metadata to get X)
metadata_cols = ['nct_id', 'official_title', 'why_stopped', 'overall_status', 'start_date', 'completion_date', 'brief_title', 'brief_summary_text', 'detailed_description_text', 'txt_criteria']
X = df.drop(columns=[c for c in metadata_cols if c in df.columns], errors='ignore')

# 1. Predict
print(">>> Generating Predictions...")
# We need to use the pipeline's transform to ensure columns match
preprocessor = model.named_steps['preprocessor']
classifier = model.named_steps['classifier']

try:
    X_trans = preprocessor.transform(df) # Pass full df, preprocessor picks columns
    probs = model.predict_proba(df)[:, 1]
    df['risk_score'] = probs
except Exception as e:
    print(f"Error in prediction: {e}")
    # Fallback: Try to predict on X directly if pipeline expects specific cols
    X_trans = preprocessor.transform(X)
    probs = model.predict_proba(X)[:, 1]
    df['risk_score'] = probs

# 2. SHAP
print(">>> Calculating SHAP Drivers...")
explainer = shap.TreeExplainer(classifier)
shap_values = explainer.shap_values(X_trans)
feature_names = preprocessor.get_feature_names_out()

# 3. Aggregate Pillars per Trial
dominant_pillars = []
for i in range(len(df)):
    scores = {'Patient & Criteria': 0, 'Protocol Design': 0, 'Sponsor & Operations': 0, 'Therapeutic Landscape': 0}
    for idx, val in enumerate(shap_values[i]):
        # We sum ABSOLUTE impact to find the "Main Topic" the model cares about
        scores[get_pillar(feature_names[idx])] += abs(val)
    dominant_pillars.append(max(scores, key=scores.get))

df['model_driver'] = dominant_pillars
df['human_reason'] = df['why_stopped'].apply(tag_real_reason)

# ====================================================================
# 3. SELECTION: THE "CASTING CALL"
# ====================================================================
print("\n" + "="*60)
print(" CASTING RESULTS: THE BEST TRIALS FOR YOUR DEMO")
print("="*60)

# --- A. THE FAILURES (Aligned Logic) ---
# We want: Terminated + High Risk Score + (Model Driver == Human Reason)
failures = df[
    (df['overall_status'] != 'COMPLETED') &
    (df['human_reason'] != 'Unknown') &
    (df['human_reason'] != 'Other') &
    (df['model_driver'] == df['human_reason']) # <--- THE ALIGNMENT CHECK
].sort_values('risk_score', ascending=False)

print(f"\n[A] TOP ALIGNED FAILURES (Found {len(failures)})")
print("These are perfect because the Model's math matches the Human's text.")

categories = ['Therapeutic Landscape', 'Sponsor & Operations', 'Patient & Criteria', 'Protocol Design']
selected_failures = []

for cat in categories:
    subset = failures[failures['human_reason'] == cat].head(20)
    if not subset.empty:
        print(f"\n--- Category: {cat} ---")
        for _, row in subset.iterrows():
            print(f"  ID: {row['nct_id']} | Risk: {row['risk_score']:.1%} | Reason: {row['why_stopped'][:100]}...")
            selected_failures.append(row['nct_id'])

# --- B. THE SUCCESSES (Diverse Drivers) ---
# We want: Completed + Low Risk Score
successes = df[
    (df['overall_status'] == 'COMPLETED') &
    (df['risk_score'] < 0.40)
].sort_values('risk_score', ascending=True)

print(f"\n\n[B] TOP SUCCESS STORIES (Found {len(successes)})")
print("These show the model correctly identifying safe/effective trials.")

selected_successes = []
for cat in categories:
    # We pick successes where the model felt that specific pillar was the STRONGEST positive signal
    subset = successes[successes['model_driver'] == cat].head(20)
    if not subset.empty:
        print(f"\n--- Driver: {cat} ---")
        for _, row in subset.iterrows():
            print(f"  ID: {row['nct_id']} | Risk: {row['risk_score']:.1%} | Title: {row['official_title'][:80]}...")
            selected_successes.append(row['nct_id'])

# ====================================================================
# 4. SAVE THE FINAL PLAYLIST
# ====================================================================
final_playlist = selected_failures + selected_successes
print(f"\n\n>>> Total Selected for Demo: {len(final_playlist)}")

# Save just these IDs to a text file for easy copy-pasting
with open('demo_playlist_ids.txt', 'w') as f:
    for nid in final_playlist:
        f.write(f"{nid}\n")

print("IDs saved to 'demo_playlist_ids.txt'")

>>> Loading Data & Model...
>>> Generating Predictions...
>>> Calculating SHAP Drivers...

 CASTING RESULTS: THE BEST TRIALS FOR YOUR DEMO

[A] TOP ALIGNED FAILURES (Found 199)
These are perfect because the Model's math matches the Human's text.

--- Category: Therapeutic Landscape ---
  ID: NCT03911869 | Risk: 81.3% | Reason: Study terminated due to lack of enrollment of the target population. There were no safety, efficacy,...
  ID: NCT05144009 | Risk: 80.9% | Reason: The benefit-risk profile does not support continuation of the LOTIS-9 trial....
  ID: NCT05256381 | Risk: 80.1% | Reason: Due to lack of expected efficacy shown at the time of the interim analysis...
  ID: NCT03690141 | Risk: 79.7% | Reason: Terminated early due to a lack of efficacy....
  ID: NCT05267106 | Risk: 78.8% | Reason: Recruitment ceased after a pre-planned futility interim analysis indicated a low probability to conf...
  ID: NCT03184558 | Risk: 78.3% | Reason: Based on planned futility assessment...
  ID: NC

In [8]:
import shutil
import os

# Define paths
current_location = "demo_playlist_ids.txt"
target_location = "/home/delaunan/code/delaunan/clintrialpredict/data/demo_playlist_ids.txt"

# Move the file
if os.path.exists(current_location):
    shutil.move(current_location, target_location)
    print(f"✅ File moved to: {target_location}")
else:
    print(f"⚠️ File not found in current directory. It might already be in data/ or deleted.")

✅ File moved to: /home/delaunan/code/delaunan/clintrialpredict/data/demo_playlist_ids.txt


In [20]:
import pandas as pd
import numpy as np
import os
import csv
import re
import sys
from typing import Optional

# ====================================================================
# 0. CONFIGURATION (THE ONLY LINE YOU MUST VERIFY)
# ====================================================================
# This MUST be the absolute path to the FOLDER containing your AACT .txt files.
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
# --------------------------------------------------------------------

print(f"DEBUG: Data Path is set to: {DATA_PATH}")
print("--- STARTING PREDICTION COHORT GENERATION ---")

# ====================================================================
# 1. CLINICAL TRIAL LOADER CLASS (Full Definition)
# ====================================================================

class ClinicalTrialLoader:
    """
    A specialized loader for generating the PROSPECTIVE prediction dataset (2024-2025).
    It ensures feature consistency and excludes all post-hoc/leakage features.
    """
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.df_drugs = pd.DataFrame()

        # --- STRATEGY A: PERFECT ---
        self.params_perfect = {
            "sep": "|", "dtype": str, "header": 0, "quotechar": '"',
            "quoting": csv.QUOTE_MINIMAL, "low_memory": False, "on_bad_lines": "warn"
        }

        # --- STRATEGY B: ROBUST ---
        self.params_robust = {
            "sep": "|", "dtype": str, "header": 0, "quotechar": '"',
            "quoting": 3, "low_memory": False, "on_bad_lines": "warn"
        }

    def _safe_load(self, filename: str, cols: Optional[list] = None) -> pd.DataFrame:
        full_path = os.path.join(self.data_path, filename)
        if not os.path.exists(full_path):
            print(f"   [!] Warning: File not found {filename}. Features will be empty.")
            return pd.DataFrame()

        try:
            return pd.read_csv(full_path, usecols=cols, **self.params_perfect)
        except Exception as e:
            print(f"   [!] Formatting error in {filename}. Switching to Robust Mode...")
            try:
                return pd.read_csv(full_path, usecols=cols, **self.params_robust)
            except Exception as e2:
                print(f"   [x] CRITICAL: Could not load {filename}. Error: {e2}")
                return pd.DataFrame()

    # --- STEP 1: LOAD FULL COHORT FOR TIME-SENSITIVE FEATURES ---

    def load_full_cohort_for_competition(self, min_year: int = 2005, max_year: int = 2025) -> pd.DataFrame:
        """
        Loads the entire relevant dataset (2005-2025) to calculate time-sensitive
        features (Competition) on the complete historical context before splitting.
        """
        print(f">>> 1. Loading Full Cohort ({min_year}-{max_year}) for Competition Calculation...")

        # 1. Load Core Columns
        cols_studies = ['nct_id', 'overall_status', 'study_type', 'phase',
                        'start_date', 'number_of_arms',
                        'official_title', 'why_stopped',
                        'has_dmc', 'is_fda_regulated_drug', 'brief_title']

        df = self._safe_load('studies.txt', cols=cols_studies)
        if df.empty:
            raise ValueError("Critical Error: 'studies.txt' failed to load.")

        # 2. Apply Core Filters (Interventional, Drug, Phase 2/3)
        df = df[df['study_type'].str.upper() == 'INTERVENTIONAL'].copy()

        cols_int = ['nct_id', 'intervention_type', 'name']
        df_int = self._safe_load('interventions.txt', cols=cols_int)
        if not df_int.empty:
            target_types = ['DRUG', 'BIOLOGICAL', 'GENETIC']
            drug_ids = df_int[df_int['intervention_type'].str.upper().isin(target_types)]['nct_id'].unique()
            df = df[df['nct_id'].isin(drug_ids)]
            self.df_drugs = df_int[df_int['intervention_type'].str.upper().isin(target_types + ['DIETARY SUPPLEMENT', 'OTHER'])].copy()

        excluded_phases = ['EARLY_PHASE1', 'PHASE4', 'NA'] #'PHASE1'
        df = df[~df['phase'].astype(str).str.upper().isin(excluded_phases)]
        df = df.dropna(subset=['phase'])

        # 3. Apply Full Date Range Filter (2005-2025)
        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        df['start_year'] = df['start_date'].dt.year
        df = df[df['start_year'].between(min_year, max_year)].copy()

        # 4. Calculate Time-Sensitive Features (Hierarchy and Competition)
        df = self._attach_medical_hierarchy(df)
        df = self._calculate_competition(df)

        print(f"    Full Cohort Size: {len(df)} trials.")
        return df.copy()


    # --- STEP 2: FILTER AND FINALIZE PREDICTION SET ---

    def load_and_clean_prediction_set(self, df_full_cohort: pd.DataFrame, start_year_predict: int = 2024) -> pd.DataFrame:
        """
        Filters the full cohort for the Prediction Set: Active/Pending Statuses, Year >= 2024.
        """
        df = df_full_cohort.copy()
        print(f">>> 2. Filtering for Prediction Set ({start_year_predict}-2025, Active/Pending Statuses)...")

        # 1. Filter: Date Range (2024-2025)
        df = df[df['start_year'] >= start_year_predict].copy()

        # 2. Filter: Status (Active/Pending Only)
        # 2. Filter: Specific NCT IDs
        # (Replaces the previous Active/Pending status filter)
        TARGET_NCT_IDS = [
            'NCT03911869', 'NCT05144009', 'NCT05256381', 'NCT03690141',
            'NCT05267106', 'NCT03184558', 'NCT02850406', 'NCT03341962',
            'NCT03301636', 'NCT03901469', 'NCT01514864', 'NCT03871829',
            'NCT03844932', 'NCT02150967', 'NCT04265534', 'NCT04987307',
            'NCT03840902', 'NCT03955471', 'NCT04752358', 'NCT06964165',
            'NCT04796324', 'NCT05441826', 'NCT04873869', 'NCT04454567',
            'NCT03139032', 'NCT01963611', 'NCT01606566', 'NCT05176717',
            'NCT04560166', 'NCT03674242', 'NCT04264442', 'NCT05741346',
            'NCT04702568', 'NCT05085964', 'NCT05659459', 'NCT04781647',
            'NCT04166552', 'NCT02074410', 'NCT02456675', 'NCT03591965',
            'NCT03339401', 'NCT02859727', 'NCT02257619', 'NCT03005054',
            'NCT05177107', 'NCT01679041', 'NCT02785120', 'NCT03336242',
            'NCT03943147', 'NCT02865122', 'NCT01620268', 'NCT02128269',
            'NCT02606994', 'NCT04715230', 'NCT01750684', 'NCT01546350',
            'NCT06012565', 'NCT04480138', 'NCT02468414', 'NCT04938180',
            'NCT06091826', 'NCT02719743', 'NCT05734040', 'NCT05284799',
            'NCT04363359', 'NCT03849560', 'NCT01669122', 'NCT01561768',
            'NCT01669096', 'NCT02359201', 'NCT03992872', 'NCT04669691',
            'NCT03467412', 'NCT04192500', 'NCT01930357', 'NCT01732627',
            'NCT01943825', 'NCT03276962', 'NCT01784874', 'NCT02553343'
        ]

        # Filter to include only the specific NCT IDs listed above
        # (Ensure 'nct_id' matches the column name in your dataframe)
        df = df[df['nct_id'].isin(TARGET_NCT_IDS)]



        # 3. Filter: COVID Sanitizer (Kept for consistency)
        if 'why_stopped' in df.columns:
            covid_keywords = ['covid', 'pandemic', 'coronavirus', 'sars-cov-2', 'logistical reasons']
            mask_covid = df['why_stopped'].fillna('').astype(str).str.lower().apply(
                lambda x: any(k in x for k in covid_keywords)
            )
            if mask_covid.sum() > 0:
                print(f"    [Sanitizer] Dropping {mask_covid.sum()} trials terminated due to COVID/Logistics.")
                df = df[~mask_covid]

        # 4. Target: The prediction set has NO target.
        if 'target' in df.columns:
             df.drop(columns=['target'], inplace=True)

        print(f"    Prediction Cohort: {len(df)} trials (Active/Pending Phase 2/3, {start_year_predict}-2025)")

        # 5. Add remaining features and user-facing text
        df = self.add_features_for_prediction(df)

        return df.copy()

    # --- NEW PREDICTION FEATURE FUNCTION (STEP 3) ---

    def add_features_for_prediction(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Adds remaining features (non-time-sensitive) and user-facing fields.
        """
        df = df.copy()
        print(">>> 3. Engineering Remaining Features for Prediction...")

        # 1. Operational Flags
        df['covid_exposure'] = df['start_year'].between(2019, 2022).astype(int)

        # 2. Geography (SAFE: Only is_us)
        df_countries = self._safe_load('countries.txt', cols=['nct_id', 'name'])
        if not df_countries.empty:
            us_trials = df_countries[df_countries['name'] == 'United States']['nct_id'].unique()
            df['includes_us'] = df['nct_id'].isin(us_trials).astype(int)
        else:
            df['includes_us'] = 0

        # 3. Merge Standard Metadata
        df = self._merge_file(df, 'designs.txt', ['nct_id', 'allocation', 'intervention_model', 'masking', 'primary_purpose'])
        # NOTE: This line adds 'number_of_primary_outcomes_to_measure'
        df = self._merge_file(df, 'calculated_values.txt', ['nct_id', 'number_of_primary_outcomes_to_measure'])

        # 4. Sponsor Engineering
        df = self._engineer_sponsor_features(df)

        # 5. Complexity Engineering (Loads eligibility.txt and 'criteria' column)
        df = self._engineer_complexity(df)

        # 6. Agent Type
        df = self._engineer_agent_type(df)

        # 7. Smart Patterns (Rigor & Strictness)
        df = self._engineer_smart_patterns(df)

        # 8. Safe Features (DMC)
        df = self._engineer_safe_features(df)

        # 9. Text Features (Creates txt_tags, txt_criteria)
        df = self._prepare_text(df)

        # 10. ATTACH EMBEDDINGS (TEMPORARILY SKIPPED FOR COLLEAGUE'S WORKFLOW)
        df = self._attach_embeddings(df)

        # 11. Attach User-Facing Text Fields
        df = self._load_user_facing_text(df)

        # --- CLEANUP (FIXED LOGIC) ---
        old_col = 'number_of_primary_outcomes_to_measure'
        new_col = 'num_primary_endpoints'

        if old_col in df.columns:
            df.rename(columns={old_col: new_col}, inplace=True)
            # Convert to numeric and fill NaNs in the new column
            df[new_col] = pd.to_numeric(df[new_col], errors='coerce').fillna(1)
        else:
            # If the column was never loaded, create it with the default value 1
            df[new_col] = 1
        # -----------------------------

        return df

    # --- NEW HELPER FUNCTION FOR UI/USER-FACING DATA ---

    def _load_user_facing_text(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Loads text fields that are useful for the UI but not necessarily for the model.
        """
        print("    -> Loading User-Facing Text (Brief Summary, Detailed Description)...")

        # 1. Brief Summary
        cols_brief = ['nct_id', 'description']
        df_brief = self._safe_load('brief_summaries.txt', cols=cols_brief)
        if not df_brief.empty:
            df_brief = df_brief.rename(columns={'description': 'brief_summary_text'})
            df = df.merge(df_brief.drop_duplicates('nct_id'), on='nct_id', how='left')
        else:
            df['brief_summary_text'] = ""

        # 2. Detailed Description
        cols_detailed = ['nct_id', 'description']
        df_detailed = self._safe_load('detailed_descriptions.txt', cols=cols_detailed)
        if not df_detailed.empty:
            df_detailed = df_detailed.rename(columns={'description': 'detailed_description_text'})
            df = df.merge(df_detailed.drop_duplicates('nct_id'), on='nct_id', how='left')
        else:
            df['detailed_description_text'] = ""

        return df

    # --- ALL OTHER NECESSARY HELPER METHODS (KEPT FOR FUNCTIONALITY) ---

    def _engineer_smart_patterns(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Smart Patterns (Rigor & Strictness)...")
        def get_masking_score(val):
            val = str(val).lower()
            if 'quadruple' in val: return 3
            if 'double' in val: return 2
            if 'single' in val: return 1
            return 0
        def get_allocation_score(val):
            return 1 if 'randomized' in str(val).lower() else 0
        def get_model_score(val):
            val = str(val).lower()
            return 1 if 'crossover' in val or 'factorial' in val else 0
        df['score_masking'] = df['masking'].apply(get_masking_score)
        df['score_allocation'] = df['allocation'].apply(get_allocation_score)
        df['score_model'] = df['intervention_model'].apply(get_model_score)
        df['design_rigor_score'] = df['score_masking'] + df['score_allocation'] + df['score_model']
        df['is_gender_restricted'] = df['gender'].apply(lambda x: 0 if str(x).lower() == 'all' else 1)
        df['is_sick_only'] = df['healthy_volunteers'].apply(lambda x: 1 if str(x).lower() == 'no' else 0)
        for col in ['child', 'adult', 'older_adult']:
            if col in df.columns:
                df[col] = df[col].astype(str).apply(lambda x: 1 if x.lower() in ['true', '1', 'yes'] else 0)
            else:
                df[col] = 1
        df['eligibility_strictness_score'] = (
            df['is_gender_restricted'] +
            df['is_sick_only'] +
            (1 - df['child']) +
            (1 - df['older_adult'])
        )
        return df

    def _engineer_agent_type(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Agent Type (Bulletproof Classifier)...")
        if self.df_drugs.empty:
            df['agent_category'] = 'UNKNOWN'
            return df
        type_priority = {'GENETIC': 1,'BIOLOGICAL': 2,'DRUG': 3,'DIETARY SUPPLEMENT': 4,'OTHER': 5}
        df_int = self.df_drugs.copy()
        df_int['type_upper'] = df_int['intervention_type'].str.upper()
        df_int['priority'] = df_int['type_upper'].map(lambda x: type_priority.get(x, 5))
        df_int = df_int.sort_values('priority')
        best_types = df_int.drop_duplicates('nct_id')[['nct_id', 'type_upper', 'name']]
        df = df.merge(best_types, on='nct_id', how='left')
        def classify_molecule(row):
            itype = str(row['type_upper'])
            name = str(row['name']).lower()
            if 'placebo' in name: return 'PLACEBO_CTRL'
            if itype == 'GENETIC': return 'GENE_THERAPY'
            if any(x in name for x in ['car-t', 'chimeric antigen', 'autologous', 'allogeneic', 't-cell', 'nk cell']):
                return 'CELL_THERAPY'
            if name.endswith('cel'): return 'CELL_THERAPY'
            if any(x in name for x in ['crispr', 'cas9', 'mrna', 'sirna', 'antisense', 'oligonucleotide', 'plasmid', 'vector', 'aav']):
                return 'RNA_GENE_THERAPY'
            if itype == 'BIOLOGICAL': return 'BIOLOGIC'
            if 'mab' in name:
                if 'adc' in name or 'conjugate' in name: return 'ANTIBODY_DRUG_CONJUGATE'
                return 'MONOCLONAL_ANTIBODY'
            if name.endswith('cept'): return 'BIOLOGIC_FUSION'
            if 'vaccine' in name: return 'VACCINE'
            if any(x in name for x in ['interferon', 'interleukin', 'cytokine']): return 'IMMUNOTHERAPY'
            if name.endswith('ib') or 'ib ' in name:
                if 'tinib' in name: return 'KINASE_INHIBITOR_TYROSINE'
                if 'parib' in name: return 'PARP_INHIBITOR'
                if 'lisib' in name: return 'PI3K_INHIBITOR'
                return 'TARGETED_KINASE_INHIBITOR'
            if 'vastatin' in name: return 'STATIN_CHOLESTEROL'
            if name.endswith('stat') or 'stat ' in name: return 'ENZYME_INHIBITOR'
            if name.endswith('degib'): return 'HEDGEHOG_INHIBITOR'
            if name.endswith('clax'): return 'BCL2_INHIBITOR'
            chemo_stems = ['platin', 'taxel', 'rubicin', 'fluorouracil', 'gemcitabine',
                           'cyclophosphamide', 'methotrexate', 'etoposide', 'vincristine', 'vinblastine']
            if any(x in name for x in chemo_stems):
                return 'CHEMOTHERAPY'
            return 'SMALL_MOLECULE_OTHER'
        df['agent_category'] = df.apply(classify_molecule, axis=1)
        df.drop(columns=['type_upper', 'priority', 'name_y'], inplace=True, errors='ignore')
        if 'name_x' in df.columns: df.rename(columns={'name_x': 'official_title'}, inplace=True)
        return df

    def _engineer_safe_features(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Safe Protocol Features...")
        for col in ['has_dmc', 'is_fda_regulated_drug']:
            if col in df.columns:
                df[col] = df[col].astype(str).apply(
                    lambda x: 1 if x.lower() in ['true', 't', '1', 'yes'] else 0
                )
            else:
                df[col] = 0
        return df

    def _attach_medical_hierarchy(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Attaching Medical Hierarchy (Bridge: nct_id -> mesh_term -> area)...")
        cols_bridge = ['nct_id', 'mesh_term']
        df_bridge = self._safe_load('browse_conditions.txt', cols=cols_bridge)
        if df_bridge.empty:
            cols_cond = ['nct_id', 'name']
            df_bridge = self._safe_load('conditions.txt', cols=cols_cond)
            if not df_bridge.empty:
                df_bridge.rename(columns={'name': 'mesh_term'}, inplace=True)
        mesh_path = os.path.join(self.data_path, 'mesh_lookup.csv')
        df_dictionary = pd.DataFrame()
        if os.path.exists(mesh_path):
            try:
                df_dictionary = pd.read_csv(mesh_path, sep='|', on_bad_lines='skip')
                if 'mesh_term' in df_dictionary.columns and 'therapeutic_area' in df_dictionary.columns:
                    df_dictionary = df_dictionary[['mesh_term', 'therapeutic_area']].drop_duplicates()
                    df_dictionary.rename(columns={'therapeutic_area': 'lookup_area'}, inplace=True)
                else:
                    df_dictionary = pd.DataFrame()
            except Exception as e:
                print(f"       [!] Error reading mesh_lookup.csv: {e}")
        if not df_bridge.empty:
            if not df_dictionary.empty:
                df_full_mesh = df_bridge.merge(df_dictionary, on='mesh_term', how='left')
            else:
                df_full_mesh = df_bridge
                df_full_mesh['lookup_area'] = np.nan
            df_grouped = df_full_mesh.groupby('nct_id').agg({
                'mesh_term': 'first',
                'lookup_area': 'first'
            }).reset_index()
            df = df.merge(df_grouped, on='nct_id', how='left')
        else:
            df['mesh_term'] = np.nan
            df['lookup_area'] = np.nan
        smart_path = os.path.join(self.data_path, 'smart_pathology_lookup.csv')
        if os.path.exists(smart_path):
            try:
                df_smart = pd.read_csv(smart_path)
                if 'nct_id' in df_smart.columns:
                    df = df.merge(df_smart, on='nct_id', how='left')
            except:
                pass
        if 'best_pathology' not in df.columns: df['best_pathology'] = np.nan
        if 'therapeutic_area' not in df.columns: df['therapeutic_area'] = np.nan
        def get_final_category(row):
            if pd.notna(row.get('best_pathology')) and str(row.get('best_pathology')) != 'Unknown':
                return row['best_pathology']
            if pd.notna(row.get('lookup_area')) and str(row.get('lookup_area')) != 'Other/Unclassified':
                return row['lookup_area']
            if pd.notna(row.get('mesh_term')):
                return row['mesh_term']
            if pd.notna(row.get('therapeutic_area')) and str(row.get('therapeutic_area')) != 'Other':
                return row['therapeutic_area']
            return 'Unclassified Condition'
        df['therapeutic_subgroup_name'] = df.apply(get_final_category, axis=1)
        df.drop(columns=['mesh_term', 'lookup_area'], inplace=True, errors='ignore')
        return df

    def _engineer_sponsor_features(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Sponsor Tiers...")
        cols_needed = ['nct_id', 'lead_or_collaborator', 'name', 'agency_class']
        df_sponsors = self._safe_load('sponsors.txt', cols=cols_needed)
        if df_sponsors.empty:
            df['sponsor_tier'] = 'TIER_2_OTHER'
            df['sponsor_clean'] = 'UNKNOWN'
            df['agency_class'] = 'UNKNOWN'
            return df
        leads = df_sponsors[df_sponsors['lead_or_collaborator'].str.lower() == 'lead'][['nct_id', 'name', 'agency_class']]
        leads = leads.rename(columns={'name': 'lead_sponsor'})
        leads = leads.drop_duplicates('nct_id')
        df = df.merge(leads, on='nct_id', how='left')
        df['lead_sponsor'] = df['lead_sponsor'].fillna('UNKNOWN')
        df['agency_class'] = df['agency_class'].fillna('UNKNOWN')
        clean_col = df['lead_sponsor'].astype(str).str.lower().str.strip()
        legal_pattern = r'[.,]|\binc\b|\bltd\b|\bllc\b|\bcorp\b|\bgmbh\b|\bsa\b|\bplc\b'
        clean_col = clean_col.str.replace(legal_pattern, '', regex=True).str.strip()
        mappings = {
            'Pfizer': ['pfizer', 'wyeth', 'hospira'], 'GSK': ['glaxo', 'gsk', 'smithkline'],
            'Novartis': ['novartis', 'sandoz'], 'AstraZeneca': ['astrazeneca', 'medimmune'],
            'Merck': ['merck', 'msd'], 'Roche': ['roche', 'genentech', 'hoffmann'],
            'Sanofi': ['sanofi', 'aventis', 'genzyme'], 'J&J': ['johnson & johnson', 'janssen'],
            'Bayer': ['bayer', 'monsanto'], 'Boehringer': ['boehringer'],
            'BMS': ['bristol-myers', 'squibb', 'celgene'], 'Lilly': ['lilly'],
            'Abbott': ['abbott', 'abbvie'], 'Amgen': ['amgen'],
            'Takeda': ['takeda', 'shire'], 'Gilead': ['gilead'],
            'Novo Nordisk': ['novo nordisk'], 'NIH': ['national cancer institute', 'nci', 'national institutes of health', 'nih']
        }
        final_names = clean_col.copy()
        for std, keys in mappings.items():
            pattern = '|'.join(keys)
            mask = clean_col.str.contains(pattern, case=False, regex=True)
            final_names.loc[mask] = std
        df['sponsor_clean'] = final_names
        def get_tier(name):
            if name in mappings.keys(): return 'TIER_1_GIANT'
            return 'TIER_2_OTHER'
        df['sponsor_tier'] = df['sponsor_clean'].apply(get_tier)
        return df

    def _engineer_complexity(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Engineering Protocol Complexity (Calculating Age Flags)...")
        cols_needed = ['nct_id', 'criteria', 'gender', 'healthy_volunteers', 'minimum_age', 'maximum_age']
        df_elig = self._safe_load('eligibilities.txt', cols=cols_needed)
        if df_elig.empty:
            df['criteria_len_log'] = 0
            for c in ['gender', 'healthy_volunteers', 'adult', 'child', 'older_adult']: df[c] = 0
            return df
        df_elig = df_elig.drop_duplicates('nct_id')
        df = df.merge(df_elig, on='nct_id', how='left')
        df['criteria_len_log'] = np.log1p(df['criteria'].astype(str).str.len().fillna(0))
        df['healthy_volunteers'] = df['healthy_volunteers'].astype(str).str.lower().apply(
            lambda x: 'no' if x in ['f', 'false', '0', 'no', 'nan'] else 'yes'
        )
        df['gender'] = df['gender'].fillna('UNKNOWN')
        def parse_age_to_years(val, default_val):
            if pd.isna(val) or str(val).lower() in ['n/a', 'nan', '', 'none']: return default_val
            try:
                match = re.search(r'(\d+(\.\d+)?)', str(val))
                if not match: return default_val
                num = float(match.group(1))
                text = str(val).lower()
                if 'month' in text: num /= 12.0
                elif 'week' in text: num /= 52.0
                elif 'day' in text: num /= 365.0
                elif 'hour' in text: num /= 8760.0
                return num
            except:
                return default_val
        df['min_age_years'] = df['minimum_age'].apply(lambda x: parse_age_to_years(x, 0.0))
        df['max_age_years'] = df['maximum_age'].apply(lambda x: parse_age_to_years(x, 100.0))
        df['child'] = (df['min_age_years'] < 18).astype(int)
        df['adult'] = ((df['max_age_years'] >= 18) & (df['min_age_years'] < 65)).astype(int)
        df['older_adult'] = (df['max_age_years'] > 65).astype(int)
        df.drop(columns=['minimum_age', 'maximum_age', 'min_age_years', 'max_age_years'], inplace=True, errors='ignore')
        return df

    def _calculate_competition(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Calculating Competition...")
        try:
            req_cols = ['start_year', 'therapeutic_area', 'phase']
            if not all(col in df.columns for col in req_cols):
                df['competition_broad'] = 0
                df['competition_niche'] = 0
                return df
            grid = df.groupby(['start_year', 'therapeutic_area']).size().reset_index(name='count')
            lookup = dict(zip(zip(grid['start_year'], grid['therapeutic_area']), grid['count']))
            def get_comp(row):
                y, area = row['start_year'], row['therapeutic_area']
                return lookup.get((y, area), 0) + lookup.get((y-1, area), 0)
            df['competition_broad'] = df.apply(get_comp, axis=1)
            if 'therapeutic_subgroup_name' in df.columns:
                grid_niche = df.groupby(['start_year', 'therapeutic_subgroup_name']).size().reset_index(name='count')
                lookup_niche = dict(zip(zip(grid_niche['start_year'], grid_niche['therapeutic_subgroup_name']), grid_niche['count']))
                def get_niche(row):
                    y, sub = row['start_year'], row['therapeutic_subgroup_name']
                    return lookup_niche.get((y, sub), 0) + lookup_niche.get((y-1, sub), 0)
                df['competition_niche'] = df.apply(get_niche, axis=1)
            else:
                df['competition_niche'] = 0
        except:
            df['competition_broad'] = 0
            df['competition_niche'] = 0
        return df

    def _prepare_text(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Preparing Text Features...")
        df_keys = self._safe_load('keywords.txt', cols=['nct_id', 'name'])
        if not df_keys.empty:
            keys_grouped = df_keys.groupby('nct_id')['name'].apply(lambda x: " ".join(x.dropna().astype(str))).reset_index(name='txt_keywords')
            df = df.merge(keys_grouped, on='nct_id', how='left')
        else:
            df['txt_keywords'] = ""
        if not self.df_drugs.empty:
            int_names = self.df_drugs.groupby('nct_id')['name'].apply(lambda x: " ".join(x.dropna().astype(str))).reset_index(name='txt_int_names')
            df = df.merge(int_names, on='nct_id', how='left')
        else:
            df['txt_int_names'] = ""
        text_cols = ['official_title', 'txt_keywords', 'txt_int_names', 'criteria']
        for c in text_cols:
            if c in df.columns: df[c] = df[c].fillna("")
        df['txt_tags'] = (df['official_title'] + " " + df['txt_keywords'] + " " + df['txt_int_names'])
        if 'criteria' in df.columns:
            df['txt_criteria'] = df['criteria']
            df.drop(columns=['txt_keywords', 'txt_int_names'], inplace=True, errors='ignore')
        return df

    # NOTE: _attach_embeddings is now a placeholder function

    def _attach_embeddings(self, df: pd.DataFrame) -> pd.DataFrame:
        print("    -> Attaching Vectorized Text Embeddings...")
        emb_path = os.path.join(self.data_path, 'embeddings_with_nctid.csv')

        # 1. Check if file exists
        if not os.path.exists(emb_path):
            print("    [!] Warning: embeddings_with_nctid.csv not found.")
            return df

        # 2. LOAD AND MERGE
        try:
            # Load the embeddings CSV
            df_emb = pd.read_csv(emb_path)

            # Check if 'nct_id' is present to perform the merge
            if 'nct_id' in df_emb.columns:
                # Merge logic: Left join on nct_id
                df = df.merge(df_emb, on='nct_id', how='left')
                print(f"       -> Successfully merged embeddings (Columns added: {df_emb.shape[1] - 1})")
            else:
                print("       [!] Error: 'nct_id' column missing in embeddings file. Cannot merge.")

        except Exception as e:
            print(f"       [!] Error reading/merging embeddings: {e}")

        return df


    # NOTE: _attach_p_values is ONLY called in the original training pipeline (not in prediction)
    def _attach_p_values(self, df: pd.DataFrame) -> pd.DataFrame:
        # This function is not used in the prediction pipeline
        return df

    def _merge_file(self, df: pd.DataFrame, filename: str, cols: list, filter_col: Optional[str] = None, filter_val: Optional[str] = None) -> pd.DataFrame:
        try:
            aux = self._safe_load(filename, cols=cols + ([filter_col] if filter_col else []))
            if aux.empty: return df
            if filter_col:
                aux = aux[aux[filter_col] == filter_val].drop(columns=[filter_col])
            aux = aux.drop_duplicates('nct_id')
            return df.merge(aux, on='nct_id', how='left')
        except:
            return df

    def save(self, df: pd.DataFrame, filename: str = 'data_predict.csv'):
        out_path = os.path.join(self.data_path, filename)
        df.to_csv(out_path, index=False)
        print(f">>> Saved {len(df)} rows to {out_path}")

# ====================================================================
# 2. EXECUTION CODE (The part that runs the process)
# ====================================================================

try:
    # 1. Instantiate the loader
    loader = ClinicalTrialLoader(data_path=DATA_PATH)

    # --- STEP 1: Load and Calculate Time-Sensitive Features on the Full Cohort (2005-2025) ---
    df_full = loader.load_full_cohort_for_competition(min_year=2005, max_year=2025)

    # --- STEP 2: Generate Prediction Set (2024-2025, Active/Pending) ---
    # Note: start_year_predict=2024 enforces the new requirement
    df_predict = loader.load_and_clean_prediction_set(df_full, start_year_predict=2015)

    # 3. Save the final dataset for prediction
    loader.save(df_predict)

    print("\n[SUCCESS] Prediction cohort ready for model scoring and embedding generation.")

except ValueError as e:
    print(f"\n[CRITICAL FAILURE] Data loading stopped: {e}")
    print("ACTION REQUIRED: Check if 'studies.txt' is present in the DATA_PATH folder.")
except Exception as e:
    print(f"\n[UNEXPECTED FAILURE] An unexpected error occurred: {e}")
    print("ACTION REQUIRED: Check file paths, file names, and data integrity.")

DEBUG: Data Path is set to: /home/delaunan/code/delaunan/clintrialpredict/data
--- STARTING PREDICTION COHORT GENERATION ---
>>> 1. Loading Full Cohort (2005-2025) for Competition Calculation...
    -> Attaching Medical Hierarchy (Bridge: nct_id -> mesh_term -> area)...
    -> Calculating Competition...
    Full Cohort Size: 137724 trials.
>>> 2. Filtering for Prediction Set (2015-2025, Active/Pending Statuses)...
    Prediction Cohort: 62 trials (Active/Pending Phase 2/3, 2015-2025)
>>> 3. Engineering Remaining Features for Prediction...
    -> Engineering Sponsor Tiers...
    -> Engineering Protocol Complexity (Calculating Age Flags)...
    -> Engineering Agent Type (Bulletproof Classifier)...
    -> Engineering Smart Patterns (Rigor & Strictness)...
    -> Engineering Safe Protocol Features...
    -> Preparing Text Features...
    -> Attaching Vectorized Text Embeddings...
       -> Successfully merged embeddings (Columns added: 20)
    -> Loading User-Facing Text (Brief Summary, De

In [22]:
import pandas as pd
import os

# Define paths
csv_path = 'data/data_predict.csv'
excel_path = 'data/data_predict.xlsx'  # .xlsx is the modern Excel format

# Check if file exists
if os.path.exists(csv_path):
    print(f"Reading {csv_path}...")

    # Read the CSV
    df = pd.read_csv(csv_path)

    # Save as Excel
    print(f"Converting to {excel_path}...")
    df.to_excel(excel_path, index=False, engine='openpyxl')

    print("Done! File saved successfully.")
else:
    print(f"Error: Could not find {csv_path}")

Reading data/data_predict.csv...
Converting to data/data_predict.xlsx...
Done! File saved successfully.
